# **Computational Theory Assessment**

## **Introduction**

This project presents a complete, step by step implementation of the SHA-256 (Secure Hash Algorithm 256-bit) cryptographic hash function, following the specifications detailed in FIPS PUB 180-4 (Federal Information Processing Standards Publication 180-4), the official Secure Hash Standard published by the National Institute of Standards and Technology (NIST).
Rather than treating SHA-256 as a black box or relying on existing library implementations, this project systematically builds the algorithm from its most fundamental components—individual bitwise operations—up through the complete hash computation pipeline, ultimately demonstrating both the power and limitations of cryptographic hashing in real-world security applications.

**Purpose and Motivation**

Cryptographic algorithms are often used without a deep understanding of their internal mechanisms. This project addresses that gap by:

1. Demystifying cryptography: Breaking down a complex, industry-standard algorithm into understandable components
2. Understanding security properties: Revealing why specific design choices create cryptographic strength
3. Demonstrating practical implementation: showing how mathematical specifications translate into working code
3. Providing security awareness: Showing both proper and improper uses of cryptographic primitives
4. Educational depth: Providing a complete learning path from theory to practice

**Real-world significance** 

SHA-256 is not only an academic exercise — it is fundamental to modern digital security and is commonly implemented in the following fields:

* Blockchain and Cryptocurrency: Bitcoin and many cryptocurrencies use SHA-256 for proof-of-work
* Digital Signatures: RSA-SHA256 and ECDSA-SHA256 authenticate software, documents, and communications
* TLS/SSL: HTTPS connections use SHA-256 in certificate verification and key derivation
* Data Integrity: File verification, backup validation, and tamper detection
* Password Hashing: While SHA-256 alone is insufficient (as we'll demonstrate), it's a component in proper password hashing schemes

**Structure**

To properly cover all aspects of the SHA-256 algorithm, this notebook is divided into five sections: 

* Section 1 covers the fundamental operations that SHA-256 requires such as Maj, Ch, and the Sigma/sigma functions. 
* Section 2 covers the generation of cryptographic K constants (K₀–K₆₃) which are used in SHA-256's compression function. 
* Section 3 covers message preprocessing, which ensures that a message can be correctly processed by SHA-256. 
* Section 4 covers SHA-256's main implementation and performs the actual cryptographic transformation. 
* Section 5 covers a security analysis via a password cracking demonstration, utilising the SHA-256 lagorithm implemented in previous sections.
* And finally a conclusion covers the findings and reflections of this project.

#### Imports used - NumPy

NumPy is a python library that supplies efficient numerical operations, multi-dimensional arrays, and mathematical functions, and is used throughout this notebook to perform fast calculations, handle arrays of data, and implement the SHA-256 algorithm efficiently.

In [175]:
# imports 
import numpy as np
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'ignore', 'under': 'ignore', 'invalid': 'warn'}

## **Problem 1: Binary Words and Operations**

**Brief:** Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard (see page 10). Document each function with a clear docstring, explain its purpose and behaviour in Markdown, and test it with appropriate examples to verify correctness.

### **Problem 1 Introduction:**

The Secure Hash Standard (FIPS 180-4) defines a family of cryptographic hash algorithms, including **SHA-224, SHA-256, SHA-384,** and **SHA-512**. These algorithms use carefully designed bitwise operations to achieve diffusion and non-linearity, which are critical for cryptographic security.
This section of the Problems.ipynb document provides detailed documentation for the core bitwise operations used in the SHA-256 cryptographic hash algorithm, as specified in FIPS PUB 180-4 (Secure Hash Standard). The functions in this section (Problem 1) implement the operations which form the fundamental building blocks of SHA-256's compression function and message schedule expansion, as defined in the Secure Hash Standard.

**SHA-256 Core Functions - Comprehensive Documentation** 

SHA-256 achieves cryptographic security through:

* Diffusion: Small input changes cause large output changes
* Non-linearity: Complex relationships between inputs and outputs
* Avalanche effect: One bit change affects approximately 50% of output bits

The functions in this section implement the following primitives as defined in **Section 4 of the Secure Hash Standard**:

* u_32: Type conversion which ensures correct 32-bit arithmetic
* Ch, Maj, Parity: Logical functions which provide non-linear mixing
* Sigma functions: Rotation and shift operations which provide diffusion

### **u_32(x) Function**
**Purpose:**
Enforces 32-bit unsigned integer arithmetic required by the SHA-256 specification - essentially this function ensures all arithmetic and bitwise operations stay within 32-bit unsigned integer bounds. All SHA-256 operations must use modulo 2^32 arithmetic to ensure consistent, reproducible hash values across all platforms and implementations- essentially, all SHA operations must use unsigned 32-bit integers to produce correct hash values

**Why This Is Essential:** Python’s integers are unbounded by default, which would produce incorrect SHA-256 hashes. SHA-256 uses modulo 2^32 arithmetic for every operation which forces values into a 32-bit type. 

This function ensures:

* Correct overflow behavior: Values wrap around at 2^32−1 (4,294,967,295)
* Proper masking: All values stay within 32-bit bounds
* Platform consistency: Same results on all systems
* Specification compliance: Matches FIPS 180-4 requirements exactly

**Usage in SHA-256**

This function is the foundation of all SHA-256 operations:

* Applied to inputs before any bitwise operation
* Applied to intermediate results during computation
* Applied to final outputs to ensure correct format
* Must be used consistently throughout the algorithm

Below, the u_32(x) function implements this logic by using the NumPy library's uint32 type to convert a value to an unsigned 32-bit integer. It’s used here to ensure that integer values are:
* exactly 32 bits wide
* non-negative

As described in NumPy's official documentation, uint32 stores exactly 32 bits, meaning it can represent values from 0 to 2^32−1 (4,294,967,295). NumPy ensures that a value x is in the representable range (a 32-bit unsigned integer) by applying modulo 2^32 arithmetic, which means forcing a number to fit into 32 bits by keeping only its lowest 32 bits if the value goes out of range (i.e. once x goes past 2^32 − 1). Essentially, for any integer x, applying modulo 2^32 means computing: x mod 2^32, which gives the remainder after dividing x by 2^32. The result is always in the range: 0 ≤ x mod 2^32 < 2^32

Specific cases:
* If x is a negative value: NumPy applies modulo 2^32 wrapping. −1 wraps to 2^32 − 1 (4,294,967,295)
* If x is larger than 2^32 − 1, NumPy discards higher-order bits. 2^32 + 10 wraps to 10


In [176]:
def u_32(x):
    """
    Purpose: convert a value to an unsigned 32-bit integer using NumPy's uint32 data type
    
    Parameters
    ----------
    x : int
        Input integer value to be converted to an unsigned 32-bit integer
        
    Returns
    -------
    np.uint32
        Value of x when converted to an unsigned 32-bit integer with modulo 2^32 wrapping
    """

    # returns the value of x converted to an unsigned 32-bit integer using uint32 
    return np.uint32(x)

### **Parity(x, y, z) Function**

**Purpose** 

Parity is a simple nonlinear combination used in some hashing and cryptographic constructions (although SHA-256 does not use Parity, it appears in SHA-1). It serves as a symmetric bitwise mixing function by computing x ⊕ y ⊕ z using the XOR bitwise operator, meaning each output bit is 1 if an odd number of inputs have that bit set. Essentially, the Parity function determines whether an odd or even number of bits are set at each bit position across three 32-bit inputs.

For each bit position:

* Output is 1 if an odd number (1 or 3) of inputs have 1
* Output is 0 if an even number (0 or 2) of inputs have 1

**Mathematical Definition:** Parity(x, y, z) = x ⊕ y ⊕ z

**Usage Context**

This Parity operation is used within the SHA-1 algorithm to introduce mixing and complexity during the hash computation process in SHA-1 rounds 20-39 and 60-79. Parity is not used SHA-256. It is included here for:

* Completeness: Understanding the SHA family
* Comparison: Contrast with SHA-256's Ch and Maj functions
* Educational value: Demonstrates simpler non-linear mixing

SHA-256 replaced Parity with the more complex **Ch** and **Maj** functions (defined later in this section) for stronger security.

**Implementation**

The Parity operation is implemented as a function Parity(x, y, z) in this section (Problem 1), and uses the bitwise XOR operation, as it is defined in the official python documentation, to determine whether an odd or even number of bits are set at each bit position across three 32-bit inputs (x, y, z). Parity(x, y, z) applies the XOR operation twice as follows:

* Step 1: XOR compares corresponding bits of x and y: Outputs 1 if bits differ (one 1, one 0), and outputs 0 if bits match (both 0 or both 1). This result represents partial parity for x and y: Bit is 0 -> even number of 1s (0 or 2), Bit is 1 -> odd number of 1s (1)
    
* Step 2: XOR then compares this result with z: Outputs 1 if bits differ, and outputs 0 if bits match
    
* Step 3: Final result shows parity across all three inputs: Bit is 1 → odd number of 1s (1 or 3), Bit is 0 → even number of 1s (0 or 2)

Each bit of the final result tells you the parity of the corresponding bits of x, y, and z. It creates a parity check: the final bit is 1 if an odd number of input bits are 1.

**Relationship to Other Functions**

* Simpler than Ch: Treats all inputs symmetrically (no selector)
* Simpler than Maj: No majority voting, just XOR
* Pure XOR: Fully reversible and linear
* Weaker security: This is why SHA-256 uses Ch and Maj instead

In [177]:
def Parity(x, y, z):
    """
    Purpose: compute the Parity function used in SHA-1.
   
    This function performs bitwise XOR on three inputs to determine parity:
    returns 1 when an odd number of corresponding bits are set, 0 otherwise.

    Parameters
    ----------
    x, y, z : int
        32-bit integer values
        
    Returns
    -------
    np.uint32 
        Result of x ⊕ y ⊕ z
    
    """
    # returns the unsigned 32-bit integer result of the XOR of x, y, and z, which are converted to unsigned 32-bit integers using u_32() helper function
    return u_32(x) ^ u_32(y) ^ u_32(z)

### **Ch(x, y, z) — Choose Function**

**Definition (FIPS 180-4):** Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)

**Purpose**
Ch implements SHA-256's conditional selection function where x acts as a selector to "choose" bits from either y or z. This creates complex, non-linear relationships essential for cryptographic security.

**Implementation**

The Ch(x, y, z) function implements the Choose (Ch) function as defined in the Secure Hash Standard FIPS 180-4. It computes the conditional selection of a 32-bit value. The Ch() function chooses bits from either of the two input integers y and z, based on the value of input x. Essentially, this function uses x as a selector to choose bits from either y or z: 

* if a bit in x is 1, then the corresponding bit from y is chosen
* if a bit in x is 0, then the corresponding bit from x is chosen.

In the implementation of the Ch(x, y, z) function, this is done by using the AND bitwise operation on x and y, then using the NOT operation on x, before applying AND to x and z.

* First, AND compares each bit in both values being considered and only outputs 1 when both bits are 1. If only one bit is 1 or neither bits are 1, a 0 is output. Therefore where x has 1s, the corresponding bits from y are output, and where x has 0s, 0s are output. 
* Next, When performing the AND operation on x and z, the NOT operation is first used to flip all the bits in x (1→0, 0→1). When AND is performed on (NOT)x and z, the bits where x originally had 0s (now 1s) outputs the corresponding bits of z, and the bits where x originally had 1s (now 0s) outputs 0s. 
* Finally, the XOR bitwise operation is performed on the resulting values of x AND y, and (NOT)x and z. XOR is used to compare two values by outputing 1 when bits are different. Therefore when XOR is applied, we get a binary number which shows which bits are different the results of both AND operations performed on x y and x z. However, due to the fact that NOT was applied when calculating the result of x and z, but not when calculating the result of x and y, this results in the outputs of these operations being mutually exclusive - there is no overlap between the two: each result has 1s where the other has 0s and vice versa. So, in this situation, XOR simply acts like OR and combines the two results. XOR is used because...

As stated in the Secure Hash Standard, this produces an ouput which is very difficult to reverse-engineer as it depends on all three inputs in a complex way: small changes in x can cause the function to switch between y and z. This creates the avalanche effect crucial for cryptographic security.

**Usage in SHA-256** 

Used in the main compression function, rounds 0-63, where its applied to working variable e in each round

Purpose in algorithm: T1 = h + Σ₁(e) + Ch(e, f, g) + K[t] + W[t]

Where e, f, g are working variables: Ch selects between f and g based on e, which creates complex dependencies between state variables

**Cryptographic Properties**

* Non-linearity: Output depends on all three inputs in complex ways
* Avalanche effect: Small changes in x can completely change the final output (switches between y and z)
* Asymmetry: x has special role as selector (unlike Maj where all inputs are equal)
* Differential resistance: Prevents attackers from predicting how bit changes propagate

**Relationship to Other Functions**

* Different from Maj: Ch treats inputs asymmetrically (x is selector)
* Different from Parity: Ch is more complex, provides stronger mixing
* Complements Maj: SHA-256 uses both for different purposes
* Works with Σ₁: Applied together in compression function

In [178]:
def Ch(x, y, z):
    
    """
    Purpose: compute the Choose (Ch) function used in SHA-256.

    The Ch function uses x as a bitwise selector to choose bits from either y or z,
    creating non-linear mixing essential for SHA-256's security.

    Parameters
    ----------
    x, y, z : int  
      32-bit integer values
        
    Returns
    -------
    np.uint32 
      Result of (x ∧ y) ⊕ (¬x ∧ z) converted into an unsigned 32-bit integer 
    
    """

    # the result of x AND y is combined (using XOR) with the result of (NOT) x AND z
    # the result is converted to an unsigned 32-bit integer value and returned
    return u_32((u_32(x) & u_32(y)) ^ (~u_32(x) & u_32(z)))

### **Maj(x, y, z) — Majority Function**

**Purpose** 

Maj (the majority function), as defined in the Secure Hash Standard, is a bitwise Boolean function which makes up part of the SHA-256 compression function, where it is used in each round to update the hash state. It is used to mix bits in a non-linear and balanced way by outputting the majority bit of three input words at each bit position which improves resistance to cryptanalysis. This is because the majority function is balanced and treats all inputs equally, but changing one input only changes the output where that input is the minority, and the relationship between inputs and outputs is not a simple and linear combination which makes it harder to decrypt. 

**Mathematical Definition:** Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)

**Implementation**

The Maj(x, y, z) function implements SHA-256's majority voting function Maj where each output bit represents the majority value (0 or 1) among the three input bits at that position:

* If 2 or more inputs have 1 → output is 1
* If 2 or more inputs have 0 → output is 0

Essentially what it does is compare each of the corresponding bits in x, y, and z and if 2 or more of the inputs have a 1, the output is 1. If 2 or more of the inputs have a 0, the output is 0. In other words: the bit value that appears most frequently is output - hence majority vote. To achieve this majority vote, the Maj(x, y, z) function (implemented below) follows these steps:

* Step 1: the AND bitwise operation is performed on (x and y), (x and z), and (y and z) respectively. For each pair, each corresponding bit in the two values are compared: if the corresponding bits are the same, 1 is output. If they are different, 0 is output. This is essentially where the two values "vote together" for 1.
* Step 2: the first two results of these operations are combined using XOR: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z), meaning each corresponding bit in the three results are compared and if the bits are the same, then 0 is output. If one or more bits are different, then the output is 1. 

Essentially XOR outputs 1 when there's an odd number of 1s in its inputs:

* Majority is 1 (2+ ones) → Odd pairs agree (1 or 3) → XOR outputs 1 
* Majority is 0 (2+ zeros) → Even pairs agree (0) → XOR outputs 0 

**Usage in SHA-256**

Maj is used in the main compression function, all 64 rounds and is applied to working variable a in each round

Purpose in algorithm: T2 = Σ₀(a) + Maj(a, b, c)

Where: a, b, c are working variables that Maj combines through majority voting, where the result updates the state in next round

* Ch: Applied to variables (e, f, g)
* Maj: Applied to variables (a, b, c)

**Cryptographic Properties**

* Symmetric: All three inputs treated equally (no special roles)
* Balanced: Output evenly distributed (50% ones, 50% zeros)
* Non-linear: Relationship between inputs and outputs is complex
* Smooth transitions: Changing one input affects output only where it's in minority
* Democratic: Consensus-based, no single input dominates

**Relationship to Other Functions**

* Different from Ch: Maj treats all inputs equally, meaning it is symmetric, while Ch is asymmetric
* Different from Parity: Maj does majority voting, not just XOR as XOR alone is linear and vulnerable to attacks

SHA-256 uses both Ch and Maj for complete mixing because they provide different types of non-linearity: 
* Ch: Asymmetric, selector-based (x controls output)
* Maj: Symmetric, voting-based (all equal)


Works with Σ₀: as it will be discussed later in this section, Maj and Sigma0 are applied together in the compression function as using both ensures complete mixing of all working variables so there are no patterns or symmetries attackers can exploit, therefore rendering the result resistant to various cryptanalytic techniques

In [179]:
def Maj(x, y, z):
    """
    Purpose: compute the Majority (Maj) function used in SHA-256.
    
    Maj operates on three 32-bit value inputs (x, y, z) and outputs a single 32-bit value,
    which is the the result of a combination of bitwise AND and XOR operations used to determine “majority” 
    value among the three inputs.
    - If 2 or more inputs have 1 → output 1 (majority votes 1)
    - If 2 or more inputs have 0 → output 0 (majority votes 0)
    - The bit value appearing most frequently wins
    
    Parameters
    ----------
    x, y, z : int 
        32-bit integer values
        
    Returns
    -------
    np.uint32
        Result of (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
        
    """

    # returns the result of the XOR operation which combines the results of the AND operation performed on the unsigned 32-bit integers:
    # (x and y), (x and z), and (y and z)
    return u_32((u_32(x) & u_32(y)) ^ (u_32(x) & u_32(z)) ^ (u_32(y) & u_32(z)))

### **rotr(x, n) — Right Rotation**

**Purpose:** perform a right rotation on a 32-bit value. rotr(x, n) rotates bits instead of shifting in zeros which is a core operation in SHA-256's mixing steps. rotr(x, n) is different from a logical shift because instead of bits being dropped once they go out of range, the rotation wraps around the dropped bits and adds them back onto the beginning of the value. This preserves all bit information while performing a diffusion to improve encryption, which is the main function of rotr(x, n). Rotations are a core building block of the **Σ** and **σ** functions (implemented later in this section), so this helper function rotr(x, n) (shown below) implements rotations to facilitate these functions.

**Why rotations matter:**
* They are nonlinear with respect to shifts
* They ensure every output bit depends on multiple positions of the input
* They preserve all bits (unlike shifts, which lose information)

**Implementation**

The function rotr(x, n) implements the rotation as follows: **(x≫n)∣(x≪(32−n))**

The rotr(x, n) implements the operation ROTR^n(x), as defined in the Secure Hash Standard, which is a circular shift where all bits are shifted to the right and added back onto the beginning when they go out of range. This is how the function achieves this:

* The bits which are shifted out of range off the right end of the integer, using right shift (>>), wrap around to the left and are added back onto the beginning of the integer using by combining the right shift (>>) with a left shift (<<) operation using the OR bitwise operation "|". 
* This allows for all bits which are shifted off the right end to be added back onto the left end of the 32-bit value. This preserves all bit information while performing a diffusion to improve encryption

In [180]:
def rotr(x, n):
    """
    Purpose: perform a right rotation on a 32-bit value

    Parameters
    ----------
    x : int 
        32-bit integer value to rotate
    n : int 
        Number of positions to rotate right
        
    Returns
    -------
    u_32((x >> n) | (x << u_32(32 - n))): 
        The rotated value of x by n bits converted to an unsigned 32-bit integer

    """

    # convert x to an unsigned 32-bit integer using u_32() helper funciton
    x = u_32(x)

    # return the 32-bit integer value of the circular right rotation of x by n bits
    return u_32((x >> n) | (x << u_32(32 - n)))

### **Sigma Functions**

The Secure Hash Standard defines four Sigma/sigma function which are used in the SHA-256 compression function to introduce strong diffusion and ensure that small changes in input values result in widespread changes in the internal state of the hash function which is essential for cryptographic security. Essentially, by combining multiple rotated versions of the same word, the functions ensure that each output bit depends on multiple input bits. The four sigma functions operate on 32-bit words using combinations of bitwise rotations (ROTR) and logical shifts (SHR) as specified in NIST FIPS 180-4.
The sigma functions are divided into two categories:

* **Uppercase Σ (Σ₀, Σ₁):** Used in the compression function to mix the working variables.
* **Lowercase σ (σ₀, σ₁):** Used in the message schedule expansion to spread message bits across rounds.

All sigma functions operate on unsigned 32-bit integers and must be evaluated using modulo 2^32 arithmetic, as required by the Secure Hash Standard.

#### **Sigma0(x) — Uppercase Σ₀**

Definition (FIPS 180-4): **(x)=ROTR^2(x) ⊕ ROTR^13(x) ⊕ ROTR^22(x)**

**Purpose:** Σ₀ is a function used in the SHA-256 compression function to non-linearly mix the working variable a. It is applied repeatedly inside the compression function.Its purpose is to provide strong diffusion during each round of compression, ensuring that changes to the internal state propagate quickly and unpredictably.

**Implementation** 

The Sigma0(x) function implements the Sigma0 operation as defined in the Secure Hash Standard FIPS 180-4. Sigma0 combines three different rotations in order to achieve strong bit diffusion. It does this by applying three different rotations to the input variable x using ROTR^n(), where n represents the number of rotations performed on the input value x, and then combines the results of these rotations using XOR. The Sigma0 function below implements this functionality with the following steps:

* Step 1: the first rotation ROTR^2, rotates the bits in x by 2: meaning each bit in x is shifted to the right by 2, and the bits that go out of range on the right side of the x value are wrapped around and added back onto the left side of the x value, therefore ensuring a circular right rotation where no bits are lost.
* Step 2: The second rotation used on x is ROTR^13, which shifts the bits in x by 13 and ensures that the values that go out of range are wrapped onto the beginning of x to preserve all bits. 
* Step 3: The third rotation used on x is ROTR^22, which shifts the bits in x by 22 and ensures that the values that go out of range are wrapped onto the beginning of x to preserve all bits. 
* Step 4: The results of these three rotations are combined using XOR. This compares all corresponding bits in all values and outputs 1 if there is an odd number of 1s present (1 or 3) at this bit across the three rotations and outputs 0 if there are an even number of 1s. 

As stated in the Secure Hash Standard, the implementation of Sigma0 combines three different rotations to provide strong bit diffusion in the working variable updates. The specific rotation amounts (2, 13, 22) were chosen to maximize diffusion and resist cryptanalysis, as the rotation amounts sum to 37, which is prime, ensuring good bit distribution across multiple rounds. 

**Usage in SHA-256**

The Secure Hash Standard specifies Σ₀ as part of the computation of the temporary value: **T2​=Σ0​(a)+Maj(a,b,c)**


In [181]:
def Sigma0(x):
    """
    Purose: compute the Sigma0 (Σ₀) function for SHA-256 compression
       
    Parameters
    ----------
    x : int
        32-bit integer value
        
    Returns
    -------
    np.uint32: 
        Result of ROTR^2(x) ⊕ ROTR^13(x) ⊕ ROTR^22(x)

    """
    
    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)
    
    # returns the result of performing 3 rotations of 2, 13, and 22 on input x using helper method rotr() and then extracting a final value with XOR
    return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)

#### **Sigma1(x) — Uppercase Σ₁**
Definition (FIPS 180-4): **(x)=ROTR^6(x) ⊕ ROTR^11(x) ⊕ ROTR^25(x)**

**Purpose:**

Σ₁ is a function used in the SHA-256 compression function to non-linearly mix the working variable e in each round of SHA-256's main compression loop. Like Σ₀, it is applied repeatedly inside the compression function and it is intended to work complementarily with Σ₀ to provide enhanced bit diffusion during each round of compression, ensuring that changes to the internal state propagate quickly and unpredictably.

**Implementation**

This function implementing Sigma1 follows the same logic as the function implementing Sigma0: three different rotations are applied to the input variable x using the helper function rotr(x, n), where n represents the number of rotations performed on the input value x, and then the results of these rotations are combined using XOR. Sigma1 implements this functionality as follows:

* Step 1: The first rotation ROTR^6 is applied to x - it rotates the bits in x by 6
* Step 2: The second rotation used on x is ROTR^11, which shifts the bits in x by 11
* Step 3: The third rotation used on x is ROTR^25, which shifts the bits in x by 25
* Step 4: The results of these three rotations are combined using XOR. This compares all corresponding bits in all values and outpus 1 if there is an odd number of 1s present (1 or 3) at this bit across the three rotations and outputs 0 if there are an even number of 1s. 

The rotation amounts (6, 11, 25) differ from Sigma0 to ensure different diffusion patterns. These amounts 6, 11, and 25, were specifically chosen by NIST through extensive cryptanalysis to resist known attack vectors.

**Usage in SHA-256** 

It provides diffusion and non-linearity during the update of the internal state and is part of the computation of: T1​=h+Σ1​(e)+Ch(e,f,g)+Kt​+Wt​

This ensures that the evolution of the state depends on multiple rotated views of e.


In [182]:
def Sigma1(x):
    """
    Purpose: compute the Sigma1 (Σ₁) function for SHA-256 compression
    
    Parameters
    ----------
    x : int 
        32-bit integer value
        
    Returns
    -------
    np.uint32: 
        Result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
        
    """

    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)

    # return the result of performing 3 rotations of 6, 11, and 25, on input x using helper method rotr() and then extracting a final value with XOR
    return u_32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

#### **sigma0(x) — Lowercase σ₀**

**Purpose:** 

This function is used in SHA-256 to non-linearly mix words during the message schedule expansion. It is applied when generating words W[16] through W[63] from the original 16 words of a message block. The purpose of sigma0 is to spread bits from earlier words throughout the message schedule, enhancing diffusion so that changes in the input message affect many rounds of the hash function. Unlike the uppercase Σ functions used in the compression step, sigma0 combines rotations and a right shift, introducing asymmetry that strengthens the avalanche effect and ensures that high-order bits propagate differently than low-order bits.

**Implementation**

This function implements sigma0 as defined in the Secure Hash Standard FIPS 180-4, which is used to generate words 16-63 of the message schedule from the original 16 words of the message block. sigma0() applies two different rotations to the input variable x using ROTR^n(x), where n represents the number of rotations performed on the input value x, and then uses the right shift bitwise operation (n >> x) to shift the bits of x to the right, where n represents the number of bit shifts performed on x. Then, the three results of these operations are combined using XOR. This logic is implemented by sigma0() using the following steps:

* Step 1: ROTR^7(x) – Rotate x right by 7 bits. Bits that fall off the right end wrap around to the left, preserving all 32 bits.
* Step 2: ROTR^18(x) – Rotate x right by 18 bits. This further mixes the positions of the bits, spreading information from different parts of x.
* Step 3: SHR^3(x) – Shift x right by 3 bits. Unlike rotation, the bits that fall off the right end are discarded and zeros are inserted on the left, introducing asymmetry to increase non-linearity.
* Step 4: Combine with XOR – The three results from steps 1–3 are XORed together. Each resulting bit is 1 if an odd number of the three corresponding input bits are 1, and 0 if an even number are 1.

**Usage in SHA-256** 

sigma0 works in conjunction with sigma1 in the message schedule expansion equation to compute words W[t]: W[t] = σ₁(W[t-2]) + W[t-7] + σ₀(W[t-15]) + W[t-16]

This ensures that each bit of the original message block influences multiple rounds of the compression function, providing strong diffusion before the main 64-round compression loop.

In [183]:
def sigma0(x):
    """
    Function's purpose: compute the sigma0 function for SHA-256 message schedule
    
    Parameters
    ----------
    x : int
        32-bit integer value
        
    Returns
    -------
    np.uint32: 
        Result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)
    
    """

    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)
    
    # return the result of performing two rotations and a right shift on input x using helper method rotr() and then extracting a final value with XOR
    return u_32(rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))

#### **sigma1(x) — Lowercase σ₁**
Small mixing function using rotations and shifts: **(x)=ROTR^17(x) ⊕ ROTR^19(x) ⊕ (x≫10)**

**Purpose** 

The sigma1 function, like sigma0, is used in the SHA-256 message schedule expansion. Together, sigma0 and sigma1 are responsible for generating message schedule words W16 through W63 from the original 16 words of the input message block.
The purpose of sigma1 is to introduce diffusion and non-linearity into the message schedule so that each original message word influences many later words. This ensures that small changes in the message propagate through the entire compression process, contributing to the avalanche effect and overall cryptographic strength of SHA-256.
Unlike the uppercase Sigma functions used in the compression rounds, sigma1 includes a right shift operation. This deliberate asymmetry prevents the function from being reversible and strengthens resistance against cryptanalytic attacks.

**Implementation**

This function implements sigma1 as defined in the secure hash standard. The sigma1 function applies the same logic as sigma0 (defined previously), but alters the amount of rotations/bit shifts. In sigma1, three different bitwise transformations are applied to the 32-bit input value x and then the results are combined using XOR. This logic is implemented as follows:

* Step 1: ROTR^17(x) – Rotate x right by 17 bits. Bits that fall off the right end wrap around to the left, preserving all 32 bits.
* Step 2: ROTR^19(x) – Rotate x right by 19 bits. This further mixes the positions of the bits, spreading information from different parts of x.
* Step 3: SHR^10(x) – Shift x right by 10 bits. Unlike rotation, the bits that fall off the right end are discarded and zeros are inserted on the left, introducing asymmetry to increase non-linearity.
* Step 4: Combine with XOR – The three results from steps 1–3 are XORed together. Each resulting bit is 1 if an odd number of the three corresponding input bits are 1, and 0 if an even number are 1.

**Usage in SHA-256**

sigma1 works in conjunction with sigma0 in the message schedule expansion equation to compute words W[t]: W[t] = σ₁(W[t-2]) + W[t-7] + σ₀(W[t-15]) + W[t-16]

This ensures that later message schedule words depend on multiple earlier words in a complex, non-linear way.

In [184]:
def sigma1(x):
    """
    Function's purpose: compute the sigma1 function for SHA-256 message schedule
    
    Parameters
    ----------
    x : int
        32-bit integer value
        
    Returns
    -------
    np.uint32: 
        Result of ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    """
    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)
    # return the result of performing the following operations on input x:
    # 2 rotations of 17, and then 19, using helper method rotr() and a right shift of 10
    # the final 32-bit value result is found by using XOR to combine the three results from the operations
    return u_32(rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10))

### **How these functions work together**

Ch, Maj, Sigma0, and Sigma1, combine in each of the 64 rounds:

Simplified SHA-256 round: 

* T1 = h + Σ₁(e) + Ch(e, f, g) + K[t] + W[t]
* T2 = Σ₀(a) + Maj(a, b, c)

Update working variables

h = g

g = f

f = e

e = d + T1

d = c

c = b

b = a

a = T1 + T2

### **Problem 1 Tests**

#### ***Testing u_32() helper function***

In [185]:
print("=== Testing unsigned_32 ===")
value = u_32(np.uint32(0xffffffff) + np.uint32(1))
print("Input: 0xFFFFFFFF + 1")
print("32-bit wrapped output:", u_32(value))
print("Expected: 0x00000000\n")

=== Testing unsigned_32 ===
Input: 0xFFFFFFFF + 1
32-bit wrapped output: 0
Expected: 0x00000000



#### ***Testing Parity() function***

In [186]:
print("\n=== Testing Parity ===")
print("Input: 0b1010, 0b0101, 0b1100")
print("Output:", bin(Parity(np.uint32(0b1010), np.uint32(0b0101), np.uint32(0b1100))))
print("Expected: 0b1010 ^ 0b0101 ^ 0b1100 =", bin(0b1010 ^ 0b0101 ^ 0b1100))


=== Testing Parity ===
Input: 0b1010, 0b0101, 0b1100
Output: 0b11
Expected: 0b1010 ^ 0b0101 ^ 0b1100 = 0b11


#### ***Testing Ch() function***

In [187]:
print("\n=== Testing Ch ===")
print("If x bit = 1 → choose y bit")
print("If x bit = 0 → choose z bit\n")
print("Example:")
print("x = 0xFFFFFFFF, y = 0x12345678, z = 0x87654321")
print("Output:", hex(Ch(np.uint32(0xffffffff), np.uint32(0x12345678), np.uint32(0x87654321))))
print("Expected: y → 0x12345678")


=== Testing Ch ===
If x bit = 1 → choose y bit
If x bit = 0 → choose z bit

Example:
x = 0xFFFFFFFF, y = 0x12345678, z = 0x87654321
Output: 0x12345678
Expected: y → 0x12345678


#### ***Testing Maj() function***

In [188]:
print("\n=== Testing Maj ===")
print("Example bits: x=1, y=1, z=0 → majority is 1")
print("Example bits: x=0, y=0, z=1 → majority is 0\n")
print("Input: 0b1010, 0b1100, 0b1000")
print("Output:", bin(Maj(np.uint32(0b1010), np.uint32(0b1100), np.uint32(0b1000))))
print("Expected majority:", bin(0b1000))


=== Testing Maj ===
Example bits: x=1, y=1, z=0 → majority is 1
Example bits: x=0, y=0, z=1 → majority is 0

Input: 0b1010, 0b1100, 0b1000
Output: 0b1000
Expected majority: 0b1000


#### ***Testing rotr() helper function***

In [189]:
print("\n=== Testing rotr ===")
print("Input: x = 0b0001, n = 1")
print("Output:", bin(rotr(np.uint32(0b0001), 1)))
print("Expected: 0x80000000 bit pattern")
print("\nInput: x = 0x12345678, n = 4")
print("Output:", hex(rotr(np.uint32(0x12345678), 4)))
print("Expected:", hex(0x81234567))


=== Testing rotr ===
Input: x = 0b0001, n = 1
Output: 0b10000000000000000000000000000000
Expected: 0x80000000 bit pattern

Input: x = 0x12345678, n = 4
Output: 0x81234567
Expected: 0x81234567


#### ***Testing Sigma0() function***

In [190]:
print("\n=== Testing Sigma0 ===")
x = np.uint32(0x6a09e667)
print("Input: x =", hex(x))
print("Output:", hex(Sigma0(x)))
print("This mixes the word using 3 rotations.")


=== Testing Sigma0 ===
Input: x = 0x6a09e667
Output: 0xce20b47e
This mixes the word using 3 rotations.


#### ***Testing Sigma1() function***

In [191]:
print("\n=== Testing Sigma1 ===")
x = np.uint32(0xbb67ae85)
print("Input: x =", hex(x))
print("Output:", hex(Sigma1(x)))
print("Used every round inside SHA-256 compression.")


=== Testing Sigma1 ===
Input: x = 0xbb67ae85
Output: 0x758db092
Used every round inside SHA-256 compression.


#### ***Testing sigma0() function***

In [192]:
print("\n=== Testing sigma0 ===")
x = u_32(0x428a2f98)
print("Input:", hex(x))
print("Output:", hex(sigma0(x)))
print("This is used in the message schedule.")


=== Testing sigma0 ===
Input: 0x428a2f98
Output: 0xb332410e
This is used in the message schedule.


#### ***Testing sigma1() function***

In [193]:
print("\n=== Testing sigma1 ===")
x = np.uint32(0x71374491)
print("Input:", hex(x))
print("Output:", hex(sigma1(x)))
print("Also used in message schedule expansion.")


=== Testing sigma1 ===
Input: 0x71374491
Output: 0x4ac6db6c
Also used in message schedule expansion.


## **Problem 2: Fractional Parts of Cube Roots**
**Brief:** Use numpy to calculate the constants listed at the bottom of page 11 of the Secure Hash Standard, following the steps below. These are the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers.
1. Write a function called primes(n) that generates the first n prime numbers.
2. Use the function to calculate the cube root of the first 64 primes.
3. For each cube root, extract the first thirty-two bits of the fractional part.
4. Display the result in hexadecimal.
5. Test the results against what is in the Secure Hash Standard.



### **Problem 2 Introduction:**

We've seen how the functions from Problem 1 work in the SHA-256 compression function. What other aspects are there to the compression function? This is the subject of Problem 2: Implementing functions which generate the 64 constant values (K₀ through K₆₃) used in the SHA-256, as defined in the Secure Hash Standard. SHA-256 operates on 32-bit words and relies on round constants (K0–K63) in order to ensure maximum diffusion and unpredictability so there are no patterns that attackers could exploit and no hidden weaknesses. 

Cryptographic constants must be verifiable to earn trust, unpredictable-looking to prevent structural weaknesses, and free from suspicion to ensure confidence in the algorithm. In order to achieve this, cube roots of primes are used because they are a trustworthy source of “random-looking” values that cannot be secretly manipulate.
* Primes are neutral: They are well-known, fixed numbers with no hidden structure. Anyone can verify which primes are used.
* Cube roots look random: Taking the cube root of a prime produces an irrational number with no repeating pattern, so its bits don’t follow predictable rules.
* Fractional parts mix well: The decimal part of these cube roots appears random, making it ideal for spreading small input changes across many bits.

Since the process is public and deterministic, it is verifiable that the constants weren’t selected in order to hide weaknesses.

The 64 constants (K0–K63) are derived using the formula: **∛(pᵢ)** 

Where: pᵢ is the i-th prime number (p₀=2, p₁=3, p₂=5, ..., p₆₃=311) and ∛ denotes cube root

From the result of ∛(pᵢ), the fractional part is extracted (digits after the decimal point), and from that fractional part the first 32 bits are extracted and converted into hexidecimal. This process is repeated for each of the first 64 prime numbers, giving us the 64 constants K0–K63.

To implement this logic and calculate these constants, we have implemented four different functions to break down the process into simple steps. These four function are:

* primes(n): gets the first 64 prime numbers
* cube_roots(primes): calculates the cube roots of the first 64 prime numbers
* frac_32(): gets the fractional part of the cube roots of the first 64 prime numbers
* hex_conversion(constants): converts the fractional parts into hexidecimal

### **primes(n) Function**:

**Purpose:** This function implements the first step in the process for creating SHA-256's 64 round constants by generating the first 64 prime numbers (2 through 311).

**Implementation**

To implement this logic, this function uses a trial division algorithm with the following steps:

* Step 1: Start with the number 2
* Step 2: Assume the current number is prime
* Step 3: Test if the current number is divisible by any integer from 2 up to (but not including) the number itself
* Step 4: If no divisors are found, the candidate is prime and is added to the list of primes. If any divisor is found, the candidate is not prime and is discarded. 
* Step 5: Increment the number, i.e. move to the next number incrementally 
* Step 6: Repeat the process until all n (in this case 64) primes have been generated

**Why Primes for SHA-256**

Prime numbers are used because they:
* Are mathematically fundamental with no hidden structure
* Cannot be factored (by definition)
* Provide transparent, verifiable constant generation
* Create "nothing-up-my-sleeve" numbers (no backdoors)
* Ensure constants appear random but are reproducible

**Relation to other functions**

The primes(n) function generates the first 64 prime numbers that will be used to compute cube roots using the cube_roots(primes) function, facilitating the first step in the constant generation process. 

In [194]:
def primes(n):
    """"
    Purpose: generate the first n prime numbers
    
    This function implements a trial division algorithm to find prime numbers sequentially.

    Parameters
    ----------
    n : int
        Number of prime numbers to generate (SHA-256 uses n=64)
        
    Returns
    -------
    primes : list of int
        First n prime numbers in ascending order
        For n=64: [2, 3, ..., 311]

    """

    # list to store prime numbers
    primes = []

    # start from the first prime
    num = 2

    # start loop that runs untill all n prime numbers have been found
    while len(primes) < n:
        # flag to keep primality of numbers
        is_prime = True
        # test divisibility of current num by all integers from 2 up to (num - 1)
        for i in range(2,num):
            # If num is divisible by i, it is not a prime number
            if num % i == 0:
                # change the flag to mark num as a non-prime number
                is_prime = False
                # exit loop early once number is found to be non-prime
                break
        # check flag to see if num is prime
        if is_prime:
            # add the prime number to the list
            primes.append(num)
        # move on to the next integer to test
        num += 1
    # return the list of the first n prime numbers
    return primes

### **cube_roots(primes) Function:**

**Purpose:** This function implements the second step in the process for creating SHA-256's 64 round constants by calculating the cube roots (∛x) of the first 64 prime numbers (2 through 311), which were generated using the previous function **primes(n)**.

**Implementation**

The cube root of a number x is the value y such that y³ = x. Therefore the cube root of a number is calculated using the formula: (∛x)

To implement this logic, the cube_roots(primes) function uses NumPy's cbrt function to calculate the cube root of each value in the "primes" list. NumPy's cbrt function is vectorised, meaning it works by applying this formula: cbrt(x)=x^1/3 to every element in the primes list independently, so no explicit python loop is required. cbrt() automatically handles integers, floats, and negative values, and produces a NumPy array of floating-point values.

The cube_roots(primes) function implements this logic using the following steps: 

* Step 1: Take in a list "primes" of n prime numbers, in this case the first 64 prime numbers
* Step 2: Create an empty list "cube_roots" intended to store the calculated cube root values
* Step 3: Compute the cube roots of each prime in "primes" by calling cbrt(), and store the resulting array in "cube_roots"
* Step 4: Return the NumPy array "cube_roots" containing the cube roots of all values in the input list "primes"

**Why Primes for SHA-256**

SHA-256 uses cube roots of primes for the K constants because they produce irrational numbers which have infinite, non-repeating decimal expansions, making their fractional parts appear random while remaining reproducible. Using cube roots instead of square roots allows SHA-256 to ensure that there is no relationship between different constant sets.

**Relation to other functions**

The cube_roots(primes) function generates the first cube roots of the first 64 prime numbers, provided by the primes(n) function. The first 32 bits of the fractional part of these cube roots will be extracted using the frac_32() function, which is the next step in the constant generation process. 

In [195]:
def cube_root(primes):
    """
    Purpose: Compute the cube roots of a list of prime numbers
    
    Calculates ∛p for each prime p using NumPy's cbrt() function. 
    
    Parameters
    ----------
    primes : list of int
        List of prime numbers (typically first 64 primes)
        
    Returns
    -------
    numpy.ndarray of float64
        Cube root of each prime

    """

    # create list to store cube_roots
    cube_roots = [] 
    # calculate cube roots of values in "primes" list using NumPy's cbrt()
    cube_roots = np.cbrt(primes)
    # return the NumPy array of cube roots
    return cube_roots

### **frac_32() Function:**

**Why in SHA-256:** Using the fractional part ensures a uniform, non-repeating bit pattern for cryptographic mixing.

**Purpose:** This function implements the third and final step in the process for generating SHA-256's 64 round constants by extracting the first 32 bits of the fractional part of the cube roots of the first 64 prime numbers calculated by the cube_roots(primes) function. Essentially, this function utilises the functions implemented previously (primes(n), cube_roots(primes)) to generate the 64 K constants, each of which are used in a round of SHA-256's compression function.

**Implementation**

The function frac_32() extracts the fractional part of the cube roots using np.modf() by returning the value of a floating point value split into two parts: (fractional, integer). Next, the frac_32() function scales the fractional part to 32 bits and applies NumPy's floor() function to round each fractional value down to the nearest integer which ensures no rounding up occurs as this could change bit patterns and produce incorrect constants. Finally the 32-bit fractional part is cast to an unsigned 32-bit integer using NumPy's uint32 data type.

The function frac_32() implements this logic using the following steps:

* Step 1: Call the primes(n) function to generate the first 64 prime numbers
* Step 2: Call the cube_roots(primes) function to calculate the cube roots of the first 64 prime numbers
* Step 3: Split each cube root into its fractional part and integer part using NumPy's modf() function. The fractional parts are stored in "frac" while the integer parts are disgarded using a blank variable "_". 
* Step 4: The fractional parts are scaled, meaning the first 32 bits of each fractional part are extracted by using modulo 2^32 on the "frac" variable
* Step 5: Remove any remaining fractional bits from the values in "frac" using NumPy's floor() function
* Step 6: Convert the frac variables to 32-bit unsigned integers by using NumPy's uint32 data type
* Step 7: Returns a NumPy array containing the 64 constants derived from the fractional parts of the cube roots of the first 64 prime numbers, stored as unsigned 32-bit integers

**Usage in SHA-256**

The 64 K constants, as defined in SHA-256, are added to the working variables in each round of the SHA-256 compression function:

In round t (0 ≤ t ≤ 63): **T1 = h + Σ₁(e) + Ch(e, f, g) + K[t] + W[t]**

Each round uses a different constant K[t] to prevent patterns/symmetries in the algorithm, ensure each round has unique behavior, and maximise diffusion across all 64 rounds, which overall enhances security.

**Relation to other functions**

The frac_32() function generates the 64 K constants by utilising the cube_roots(primes) function to generate the cube roots of the first 64 prime numbers, provided by the primes(n) function.

In [196]:
def frac_32(n):
    """
    Purpose: Generate the 64 K constants of SHA-256's compression function

    This function extracts the first 32-bits of the fractional parts of the
    cube roots of the first 64 prime numbers, and converts them to unsigned
    32-bit integers. 
    
    Returns
    -------
    numpy.ndarray of np.uint32
        Array of 64 K constants as 32-bit unsigned integers

    """
    # get the first 64 prime numbers 
    p = primes(n)
    # get the cube roots of the first 64 prime numbers
    cube_roots = cube_root(p)
    # extract the fractional part of the cube roots using NumPy's modf()
    frac, _= np.modf(cube_roots)
    # generate the K constants by extracting the first 32 bits of the fractional parts using mod 2^32 and NumPy's floor()
    # convert these to unsigned 32-bit integers using NumPy's uint32
    constants = np.floor(frac * (2**32)).astype(np.uint32)
    # return the constants
    return constants 

### **hex_conversion(constants) Function:**

**Purpose:** Helper function that converts the 32-bit integer k constants generated by the functions implemented above (primes(n), cube_roots(primes), frac_32()), into 8-character hexadecimal strings. This allows us to verify the correctness of the generated constants by comparing them to the official K constants specified in the Secure Hash Standard, which are listed in hexidecimal format. 

**Implementation**

This function converts unsigned 32-bit integers into zero-padded hexadecimal strings for verification against the Secure Hash Standard K constant values.
The hex_conversion() function implements using the following steps:

* Step 1: Read in the NumPy array of 32-bit integer constants generated using the frac_32() function
* Step 2: Loop over the constants array and convert each uint32 to hex using Python's hex() function
* Step 3: Remove the '0x' prefix which is generated when converting to hex, using string slicing [2:]
* Step 4: Zero-pad to 8 characters using zfill(8) to ensure all values are 8 characters, as slicing can remove the first character of a value if its a 0
* Step 5: Return the list of constants as formatted hex strings

In [197]:
def hex_conversion(constants):
    """
    Purpose: Convert 32-bit constants to hexadecimal string representations

    This function transforms numpy uint32 array into zero-padded hexadecimal strings
    using hex conversion (hex()), slicing ([2:]), and zero-padding (zfill(8))

    Parameters
    ----------
    constants : numpy array of 32-bit unsigned integer constants
        
    Returns
    -------
    list of str
        Hexadecimal strings, each 8 characters long (no '0x' prefix)

    """

    # create array for storing converted constants
    hex_constants = []
    # loop over each constant in constants array
    for c in constants:
        # convert each constant to hexidecimal
        hex_constants.append(hex(c)[2:].zfill(8))
    # return the array of constants as hex strings
    return hex_constants

#### **The official 64 K constants as defined in the Secure Hash Standard**

In [198]:
K0 = np.array([
    "428a2f98", "71374491", "b5c0fbcf", "e9b5dba5", "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5",
    "d807aa98", "12835b01", "243185be", "550c7dc3", "72be5d74", "80deb1fe", "9bdc06a7", "c19bf174",
    "e49b69c1", "efbe4786", "0fc19dc6", "240ca1cc", "2de92c6f", "4a7484aa", "5cb0a9dc", "76f988da",
    "983e5152", "a831c66d", "b00327c8", "bf597fc7", "c6e00bf3", "d5a79147", "06ca6351", "14292967",
    "27b70a85", "2e1b2138", "4d2c6dfc", "53380d13", "650a7354", "766a0abb", "81c2c92e", "92722c85",
    "a2bfe8a1", "a81a664b", "c24b8b70", "c76c51a3", "d192e819", "d6990624", "f40e3585", "106aa070",
    "19a4c116", "1e376c08", "2748774c", "34b0bcb5", "391c0cb3", "4ed8aa4a", "5b9cca4f", "682e6ff3",
    "748f82ee", "78a5636f", "84c87814", "8cc70208", "90befffa", "a4506ceb", "bef9a3f7", "c67178f2"
], dtype=str)

### **Problem 2 Testing**

#### ***Testing primes(n)***

In [199]:
print("\n==================== TEST 1: PRIME NUMBER GENERATION ====================")
print("Purpose: Verify that the first 64 prime numbers are generated correctly.")

prime_nums = primes(64)

print("\nGenerated primes:")
print(prime_nums)
print(f"Number of primes generated: {len(prime_nums)} (expected: 64)")


==================== TEST 1: PRIME NUMBER GENERATION ====================
Purpose: Verify that the first 64 prime numbers are generated correctly.

Generated primes:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
Number of primes generated: 64 (expected: 64)


#### ***Testing cube_roots(primes)***

In [200]:
print("\n==================== TEST 2: CUBE ROOT CALCULATION ====================")
print("Purpose: Compute cube roots of the first 64 primes as required by FIPS 180-4.")

cubes = cube_root(prime_nums)

print("\nCube roots of primes:")
print(cubes)


==================== TEST 2: CUBE ROOT CALCULATION ====================
Purpose: Compute cube roots of the first 64 primes as required by FIPS 180-4.

Cube roots of primes:
[1.25992105 1.44224957 1.70997595 1.91293118 2.22398009 2.35133469
 2.57128159 2.66840165 2.84386698 3.07231683 3.14138065 3.33222185
 3.44821724 3.50339806 3.60882608 3.75628575 3.89299642 3.93649718
 4.0615481  4.14081775 4.1793392  4.29084043 4.36207067 4.4647451
 4.59470089 4.65700951 4.68754815 4.7474594  4.77685618 4.83458813
 5.0265257  5.07875308 5.15513674 5.18010147 5.30145919 5.32507402
 5.39469071 5.46255557 5.50687845 5.57205466 5.63574079 5.65665283
 5.75896522 5.77899657 5.81864787 5.83827246 5.95334181 6.06412699
 6.1001702  6.11803317 6.15344949 6.20582179 6.22308425 6.30799355
 6.35786118 6.40695858 6.45531481 6.47127363 6.51868392 6.54991162
 6.56541443 6.6418522  6.74599671 6.77516895]


#### ***Testing frac_32()***

In [201]:
print("\n==================== TEST 3: FRACTIONAL PART EXTRACTION/CONSTANTS GENERATION ====================")
print("Purpose: Separate fractional and integer parts of cube roots.")
print("Only the fractional part is used to derive SHA-256 constants.")

constants = frac_32(64)

print("\n32-bit integer K constants generated using the first 64 prime numbers:")
print(constants)


==================== TEST 3: FRACTIONAL PART EXTRACTION/CONSTANTS GENERATION ====================
Purpose: Separate fractional and integer parts of cube roots.
Only the fractional part is used to derive SHA-256 constants.

32-bit integer K constants generated using the first 64 prime numbers:
[1116352408 1899447441 3049323471 3921009573  961987163 1508970993
 2453635748 2870763221 3624381080  310598401  607225278 1426881987
 1925078388 2162078206 2614888103 3248222580 3835390401 4022224774
  264347078  604807628  770255983 1249150122 1555081692 1996064986
 2554220882 2821834349 2952996808 3210313671 3336571891 3584528711
  113926993  338241895  666307205  773529912 1294757372 1396182291
 1695183700 1986661051 2177026350 2456956037 2730485921 2820302411
 3259730800 3345764771 3516065817 3600352804 4094571909  275423344
  430227734  506948616  659060556  883997877  958139571 1322822218
 1537002063 1747873779 1955562222 2024104815 2227730452 2361852424
 2428436474 2756734187 3204031479 3

#### ***Testing hex_conversion***

In [202]:
print("\n==================== TEST 5: HEXADECIMAL CONVERSION ====================")
print("Purpose: Convert constants to hexadecimal format for comparison with FIPS values.")

hex_constants = hex_conversion(constants)

print("\nGenerated hex constants:")
print(hex_constants)

print("\nThe generated constants and official constants are equal: ", np.array_equal(K0, hex_constants))


==================== TEST 5: HEXADECIMAL CONVERSION ====================
Purpose: Convert constants to hexadecimal format for comparison with FIPS values.

Generated hex constants:
['428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5', 'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174', 'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da', '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967', '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85', 'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070', '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3', '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2']

The generated constants and official constants a

## Problem 3: Padding

**Brief:** Write a generator function block_parse(msg) that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. The function should accept a bytes object called msg. At each iteration, it should yield the next 512-bit block of msg as a bytes object. Ensure that the final block (or final two blocks) include(s) the required padding of msg as specified in the standard. Test the generator with messages of different lengths to confirm proper padding and block output.

###  **Problem 3 Introduction**

Before the SHA-256 compression function can process a message, it must be prepared in a specific way - everything needs to be the right size and format. This section details the first two aspects of this preprocessing: padding and parsing. 

To give some context, the SHA-256 compression function can process input data in fixed-size chunks of 512 bits called blocks, but messages often arrive in unpredictable/inconsistent lengths. Therefore all messages need to be processed and converted into a format that the SHA-256 compression function can read: blocks of 512 bits. To achieve this, there a two main steps to follow:

* Padding the message: extend the message to a specific length by padding with k 0s (Section 5.1.1)
* Parsing the message: divide the padded message into fixed-size (512 bits) blocks (Section 5.2.1)

**Importance** 

These two stages are essential parts of the compression function process, and are needed for the following reasons: 

* Needed for processing: Without this, short messages couldn't be processed and long messages couldn't be divided uniformly, rendering the compression function ineffective. It ensures every block is exactly 512 bits
* Ensures clear message representation: This guarantees that different messages never produce the same result after padding. 
* Encoding message length: Ensures the original length of the message is embedded into the padded data, which makes it bound to itself and preventing attacker from appending data and computing a valid hash. 
* Remains deterministic: The same message will always recieve the same padding and therefore produce the same hash. This ensures that all SHA-256 implementations are consistent/reproducible. 

**Padding:** Make the message length a specific multiple of 512 bits by applying the following process to a message that is l bits long:

* Add a "1" bit to the end of your message
* Add k zero bits, where k is chosen so that (l + 1 + k) equals 448 when you divide by 512 and look at the remainder
* Add a 64-bit representation of l (the original message length)

**Parsing:** Once padded, the message is broken down into n chunks (blocks) of 512 bits. Each 512-bit block is then subdivided into sixteen 32-bit words as: ***16 × 32 = 512***, meaning that each block i contains words: M₀⁽ⁱ⁾, M₁⁽ⁱ⁾, M₂⁽ⁱ⁾, ... M₁₅⁽ⁱ⁾.

To achieve padding and parsing, this section implements four functions which help in this process: 
 
* file_parse(): Converts a message into bits
* calculate_padding_length(message_length_bytes): Calculates the padding needed for a message
* create_padding_block(partial_block, message_length_bits): Apply the calculated padding to the message
* block_parse(msg): Parse the padded message by dividing it into n blocks of exactly 512 bits each

### **file_parse(filepath) function:**

**Purpose:** This function reads in a file in binary mode and returns its contents as a bytes object. This ensures that the message is in the right format for the rest of the preprocessing stage to work correctly. 
Binary mode is used because it reads exact bytes without interpretation which avoids any text mode issues such as encoding issues, altered line endings, and only successfully processing text files. The benefits of binary mode are as follows:

* all data is preserved exactly as it is stored 
* performs exact byte-for-byte reading with no encoding conversions
* Works with any file type

Essentially using binary mode means a file’s contents are read exactly as stored, without modification, which ensures reproducible hash values.

**Implementation**

The file_parse(filepath) function implements the logic described above by taking in a filepath as input, and opening the file using python's context manager - 'with'. This is a python function that ensures resources are managed safely and automatically, i.e. no need to close a file explicitly, the context manager does it for you. The file is then read in binary mode by using 'rb' where r = read and b = binary. Once everything is read, the bytes contained in the file are returned. The file_parse(filepath) function uses the following steps to implement this logic:

* Step 1: Read in the filepath
* Step 2: Open the file using python's context manager 'with'
* Step 3: Read the file in binary mode
* Step 4: Read all bytes store them
* Step 5: Return stored bytes 

**Usage in SHA-256**

SHA-256 operates on raw bytes, not text, therefore all messages must be converted to/read as bytes. This makes the file conversion function file_parse(filepath) an essential step in the preprocessing stage.

**Relation to other functions**

This function prepares the content so it can be properly padded/parsed by the other functions implemented in this section.

In [203]:
def file_parse(filepath):
    """
    Purpose: Read a file in binary mode and return its contents as bytes

    Parameters
    ----------
    filepath : str
        Path to the file to read
        
    Returns
    -------
    bytes
        Complete file contents as a bytes object
    """

    # Open the file using context manager 'with'
    with open(filepath, 'rb') as f: # Read the file in binary mode

        # Read all bytes from the file
        msg = f.read()
    # Automatically close the file after reading

    # Return all bytes read from file
    return msg

### **create_padded_blocks(partial_block, message_length_bits) function**

**Purpose** 

This function implements the SHA-256 message padding stage as specified in Section 5.1.1 of the Secure Hash Standard (FIPS 180-4). It is responsible for 
Its responsibility is to take the remaining bytes of a message that do not form a complete 512-bit block and generate the final padded block(s) required for SHA-256 processing. This function is implemented as a Python generator, meaning it yields padded blocks one at a time. This allows seamless integration with the message parsing stage and produces a continuous stream of valid SHA-256 input blocks.

The function ensures that the padding follows the SHA-256 specification exactly so that:

* A single '1' bit (represented as a byte 0x80) is appended immediately after the message
* The message is padded with k 0 bits where k is the smallest non-negative solution to (l + 1 + k) ≡ 448 (mod 512)
* The original message length (in bits) is appended as a 64-bit big-endian integer
* The final output consists of exactly one or two 512-bit (64-byte) blocks

**Implementation**

The create_padded_blocks() function receives the final partial block in need of padding, the original message length in bits, and the block size (64 bytes) from the block_parse() function. Based on how much space is available in the final block, it applies one of three padding scenarios defined by the SHA-256 standard:

* ***Scenario 1: Sufficient Space (≥9 bytes available)***

This scenario occurs when the partial block has at least 9 bytes of space remaining: 1 byte for 0x80 + 8 bytes for the length encoding

When it applies: Partial block is 55 bytes or less

What happens: All padding fits in a single 64-byte block

* ***Scenario 2: Tight Space (1-8 bytes available)***

This scenario occurs when the partial block has between 1 and 8 bytes of space remaining. There's room for the 0x80 byte but not enough room for the 8-byte length encoding.

When it applies: Partial block is between 56 and 63 bytes

What happens: Padding requires two 64-byte blocks

* ***Scenario 3: Perfect Fit / No Space (0 bytes available)***

This scenario occurs when the message length is exactly a multiple of 64 bytes or when the partial block is empty. There's no space in the current context for any padding.

When it applies: Message length is exactly 64, 128, 192, ... bytes (multiples of 64), i.e. the partial block extracted by block_parse() is empty 

What happens: A complete padding-only block is created

Note: This scenario also handles the empty message case (0 bytes), where the entire message consists of just this one padding block.

**Usage in SHA-256**

SHA-256 requires that:

* The total message length (after padding) is a multiple of 512 bits
* The final 64 bits encode the original message length in bits
* At least one bit of padding (the '1' bit) is always added

This function guarantees that these conditions are met. It ensures that the final padded block(s):

* Are exactly 64 bytes (512 bits) long
* Preserve the original message length in the final 8 bytes
* Contain the mandatory 0x80 padding byte
* Are valid inputs for the SHA-256 compression function

Because the function yields blocks one at a time, SHA-256 can process each padded block immediately without storing all blocks in memory. This is particularly important for: 

* Large files that would consume excessive memory if all blocks were stored
* Streaming applications where data arrives incrementally
* Memory-constrained environments

**Relation to other functions**

This function is called by block_parse(msg) after all full 64-byte blocks have been yielded. It receives input from block_parse(msg): 

* partial_block: The remaining bytes after all complete blocks (0-63 bytes)
* message_length_bits: Original message length in bits


It then applies one of three padding scenarios based on space available and yields an output to the caller of block_parse(msg) (via yield from in block_parse()): 

* One or two 64-byte padded blocks

***Integration:*** When used with yield from, its output is indistinguishable from blocks yielded directly by block_parse(), resulting in a single continuous stream of 512-bit blocks ready for SHA-256 processing.

In [204]:
def create_padded_blocks(partial_block, message_length_bits, BLOCK_SIZE):
    """
    Purpose: Create final padded block(s) for SHA-256.
    
    Parameters
    ----------
    partial_block : bytes
        Remaining message bytes (0-63 bytes)
    message_length_bits : int
        Original message length in bits
        
    Yields
    ------
    bytes
        One or two 64-byte padded blocks
    """

    remaining_space = BLOCK_SIZE - len(partial_block)
    
    # Scenario 1: At least 9 bytes available (1 for 0x80 + 8 for length)
    if remaining_space >= 9:
        padding_zeros = remaining_space - 1 - 8
        yield (partial_block + 
               bytes([0x80]) + 
               bytes([0x00] * padding_zeros) + 
               message_length_bits.to_bytes(8, byteorder='big'))
    
    # Scenario 2: Between 1 and 8 bytes available
    elif remaining_space >= 1:
        # First block: partial + 0x80 + zeros to fill
        yield partial_block + bytes([0x80]) + bytes([0x00] * (remaining_space - 1))
        # Second block: zeros + length
        yield bytes([0x00] * 56) + message_length_bits.to_bytes(8, byteorder='big')
    
    # Scenario 3: No space (empty partial block)
    else:
        yield (bytes([0x80]) + 
               bytes([0x00] * 55) + 
               message_length_bits.to_bytes(8, byteorder='big'))

### **block_parse(msg) Function**

**Purpose:** 

This function implements the message parsing stage as described in section 5.2.1 of the Secure Hash Standardand and is responsible for splitting the input message into complete 512-bit (64-byte) blocks and apply SHA-256 padding to them using the create_padded_blocks() helper function. Each block is then yielded one at a time, ready for the SHA-256 compression function. This function is implemented as a Python generator, meaning it uses python's 'yield' statement to produce values on demande instead of returning a full list after all processing is complete, which makes it very efficient for large messages. This mirrors how cryptographic hash functions operate on streams of data, as they must process data as it arrives.

**Implementation** 

To correctly split the input message into 64-byte blocks, the yield_blocks(msg, no_bytes) function uses a while loop to iterate through a message in increments of 64 bytes. Before the loop, two variables are initialised: 

* BLOCK_SIZE: initialised to the block size processed by SHA-256: 64 bytes. 
* A pointer variable 'position': which tracks the current position in the message. Essentially, it tracks how many bytes of the message have been processed so far, and increases in increments of 64 (BLOCK_SIZE). 

The while loop yields full blocks only. It does this by using a condition which ensures that only blocks that are exactly 64 bytes long are yielded. This is done by adding together the 'position' pointer (bytes) and the expected size of the next block 'BLOCK_SIZE' (64 bytes), and checking the result against the total length of the message (bytes). By doing this, the loop checks if the next block is a partial block. 

* If the loop condition is satisfied, then the 64 bytes of the message at the current position are yielded to the caller, and the position is incremented by 64 bytes (BLOCK_SIZE)

* If the loop condition is not satisfied, i.e. the addition of position + BLOCK_SIZE exceeds the message length, that means that the next block is less than 64 bytes long and therefore a partial block. In this case, the loop ends and the current block is not yielded. 

Once the while loop is broken, it updates a variable partial_block with the position of that partial block in the message. Next, processing is delegated to another generator for padding: the create_padded_blocks() function. Python's 'yield from' statement is used to iterate over all values produced by create_padded_blocks, and then yield each padded block as if it were yielded directly by block_parse. This means the caller sees a single continuous stream of blocks: both complete blocks and padded blocks in a single continuous sequence.

* Step 1: Initialise the BLOCK_SIZE to 64 bytes and message_length_bits to the total message length calculated in bits (required for SHA-256 padding)
* Step 2: Initialise the position pointer to 0 
* Step 3: Enter the parsing loop by checking whether position + BLOCK_SIZE is less than or equal to the total message length (in bytes), to guarantees that the next slice will be exactly 64 bytes long.
* Step 4: Yield a complete 64-byte block of the message using python's yield. At this point, the function pauses execution and hands the block to the caller, and resumes when the next block is requested.
* Step 5: Increment the position pointer by 64 bytes in preparation for the next iteration.
* Step 6: Exit the loop when a partial block is detected: fewer than 64 bytes remain. The partial block is not yielded at this stage.
* Step 7: Extract and store any remaining bytes in partial_block. This may be empty (if the message length is a multiple of 64
* Step 8: Delegate padding to a helper generator by passing the remaining bytes and message length to create_padded_blocks(). Python’s yield from statement is used to yield all padded blocks produced by the helper generator create_padded_blocks() as part of the same output sequence.


**Usage in SHA-256** 

SHA-256 processes messages in 512-bit blocks, one at a time. This function ensures that all full blocks are passed directly to the compression function without modification, and that the final block (or blocks) include the correct padding and length encoding. The total output is always a valid sequence of 512-bit blocks.

**Relation to other functions** 

This function is handles block parsing, and passes any partial blocks to yield_padded_blocks(...)  which applies SHA-256 padding rules and yields the final block or blocks. Together, these functions form a clear separation of responsibilities:
* Parsing full blocks
* Padding the final partial block
* Producing a single continuous stream of valid SHA-256 input blocks

In [205]:
def block_parse(msg):
    """
    Purpose: Parse message into 512-bit blocks with SHA-256 padding

    Generator function that processes a message according to FIPS 180-4
    Sections 5.1.1 (padding) and 5.2.1 (parsing). Yields complete 64-byte
    blocks, with the final block(s) including necessary padding.
    
    Parameters
    ----------
    msg : bytes
        Message to process (can be empty)
        
    Yields
    ------
    bytes
        64-byte (512-bit) blocks, including final padded block(s)
    """
    BLOCK_SIZE = 64  # 512 bits = 64 bytes
    message_length_bits = len(msg) * 8
    position = 0
    
    # Yield all complete 64-byte blocks
    while position + BLOCK_SIZE <= len(msg):
        yield msg[position:position + BLOCK_SIZE]
        position += BLOCK_SIZE
    
    # Get remaining bytes (partial block or empty if msg was multiple of 64)
    partial_block = msg[position:]
    
    # Yield final padded block(s)
    yield from create_padded_blocks(partial_block, message_length_bits, BLOCK_SIZE)

### **Problem 3 Testing**

#### Test padding scenarios

In [206]:
def test_padding_scenarios():
    """
    Test all SHA-256 padding scenarios with detailed output.
    Verifies correctness while explaining what happens internally.
    """
    print("=" * 70)
    print("SHA-256 PADDING AND PARSING TESTS")
    print("=" * 70)
    print()

    test_cases = [
        (b'', "Empty message"),
        (b'Hello', "Short message (5 bytes)"),
        (b'x' * 55, "Exactly 55 bytes (perfect fit)"),
        (b'x' * 56, "Exactly 56 bytes (boundary case)"),
        (b'x' * 63, "Exactly 63 bytes (1 byte space)"),
        (b'x' * 64, "Exactly 64 bytes (no space)"),
        (b'x' * 100, "Multiple blocks (100 bytes)"),
        (b'x' * 150, "Large message (150 bytes)"),
    ]

    for msg, description in test_cases:
        print(f"Test: {description}")
        print("-" * 70)

        blocks = list(block_parse(msg))

        print(f"Message length: {len(msg)} bytes ({len(msg) * 8} bits)")
        print(f"Number of blocks: {len(blocks)}")

        # ---------------------------------------------------------------------
        # 1. All blocks must be 64 bytes
        # ---------------------------------------------------------------------
        all_correct_size = all(len(block) == 64 for block in blocks)
        print(f"All blocks 64 bytes")
        assert all_correct_size, "All blocks must be exactly 64 bytes"

        # ---------------------------------------------------------------------
        # 2. Padding byte (0x80) must appear exactly once across all blocks
        # ---------------------------------------------------------------------
        padding_locations = []

        for block_index, block in enumerate(blocks):
            if 0x80 in block:
                padding_locations.append(
                    (block_index, block.index(0x80))
                )

        print(f"Padding byte locations: {padding_locations}")
        assert len(padding_locations) == 1, \
            "There must be exactly one 0x80 padding byte"

        padding_block_index, padding_byte_index = padding_locations[0]

        print(
            f"Padding byte found in block {padding_block_index} "
            f"at index {padding_byte_index}"
        )

        # ---------------------------------------------------------------------
        # 3. Length encoding must be in the last 8 bytes of the final block
        # ---------------------------------------------------------------------
        last_block = blocks[-1]

        encoded_length = int.from_bytes(last_block[-8:], byteorder='big')
        expected_length = len(msg) * 8

        print(f"Encoded length:  {encoded_length} bits")
        print(f"Expected length: {expected_length} bits")

        assert encoded_length == expected_length, \
            "Encoded message length is incorrect"

        # ---------------------------------------------------------------------
        # 4. Display block contents (abbreviated)
        # ---------------------------------------------------------------------
        for i, block in enumerate(blocks):
            print(f"\nBlock {i}:")
            if i == padding_block_index:
                print(
                    f"  Padding byte (0x80) at index {padding_byte_index}"
                )
            if i == len(blocks) - 1:
                print(f"  Last 8 bytes (length field): {block[-8:].hex()}")
            print(f"  First 16 bytes: {block[:16].hex()}")

        print("\n")

    print("=" * 70)
    print("ALL PADDING SCENARIO TESTS PASSED")
    print("=" * 70)


test_padding_scenarios()

SHA-256 PADDING AND PARSING TESTS

Test: Empty message
----------------------------------------------------------------------
Message length: 0 bytes (0 bits)
Number of blocks: 1
All blocks 64 bytes
Padding byte locations: [(0, 0)]
Padding byte found in block 0 at index 0
Encoded length:  0 bits
Expected length: 0 bits

Block 0:
  Padding byte (0x80) at index 0
  Last 8 bytes (length field): 0000000000000000
  First 16 bytes: 80000000000000000000000000000000


Test: Short message (5 bytes)
----------------------------------------------------------------------
Message length: 5 bytes (40 bits)
Number of blocks: 1
All blocks 64 bytes
Padding byte locations: [(0, 5)]
Padding byte found in block 0 at index 5
Encoded length:  40 bits
Expected length: 40 bits

Block 0:
  Padding byte (0x80) at index 5
  Last 8 bytes (length field): 0000000000000028
  First 16 bytes: 48656c6c6f8000000000000000000000


Test: Exactly 55 bytes (perfect fit)
-------------------------------------------------------

#### **Test empty/short messages**

In [207]:
def verify_block_parse_basic():
    print("=" * 70)
    print("BLOCK PARSE VERIFICATION – BASIC CASES")
    print("=" * 70)

    # Empty message
    blocks = list(block_parse(b''))
    assert len(blocks) == 1
    assert len(blocks[0]) == 64
    assert blocks[0][0] == 0x80
    print("✓ Empty message padding correct")

    # Short message
    blocks = list(block_parse(b'abc'))
    assert len(blocks) == 1
    assert blocks[0][:3] == b'abc'
    assert blocks[0][3] == 0x80
    length = int.from_bytes(blocks[0][-8:], byteorder='big')
    assert length == 24
    print("✓ Short message padding correct")

verify_block_parse_basic()

BLOCK PARSE VERIFICATION – BASIC CASES
✓ Empty message padding correct
✓ Short message padding correct


#### **Test Boundary Conditions (55 / 56 / 64 bytes)**

In [208]:
def verify_block_parse_boundaries():
    print("=" * 70)
    print("BLOCK PARSE VERIFICATION – BOUNDARY CASES")
    print("=" * 70)

    # 55 bytes
    blocks = list(block_parse(b'x' * 55))
    assert len(blocks) == 1
    assert blocks[0][55] == 0x80
    print("✓ 55-byte message fits in one block")

    # 56 bytes
    blocks = list(block_parse(b'x' * 56))
    assert len(blocks) == 2
    assert blocks[0][56] == 0x80
    print("✓ 56-byte message produces two blocks")

    # 64 bytes
    blocks = list(block_parse(b'x' * 64))
    assert len(blocks) == 2
    assert blocks[1][0] == 0x80
    print("✓ 64-byte message produces new padding block")

verify_block_parse_boundaries()

BLOCK PARSE VERIFICATION – BOUNDARY CASES
✓ 55-byte message fits in one block
✓ 56-byte message produces two blocks
✓ 64-byte message produces new padding block


#### **Test large/multi-block messages**

In [209]:
def verify_block_parse_large_messages():
    print("=" * 70)
    print("BLOCK PARSE VERIFICATION – LARGE MESSAGES")
    print("=" * 70)

    blocks = list(block_parse(b'x' * 200))
    assert len(blocks) == 4
    assert all(len(b) == 64 for b in blocks)
    print("200-byte message produces 4 valid blocks")

    blocks = list(block_parse(b'x' * 150))
    assert all(len(b) == 64 for b in blocks)
    print("Arbitrary large message padded correctly")

    print("\nALL LARGE MESSAGE TESTS PASSED")

verify_block_parse_large_messages()


BLOCK PARSE VERIFICATION – LARGE MESSAGES
200-byte message produces 4 valid blocks
Arbitrary large message padded correctly

ALL LARGE MESSAGE TESTS PASSED


## Problem 4: Hashes

**Brief:** Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

### **Introduction Problem 4:**

In Problem 3, we covered how SHA-256 performs its core compression operations. Problem 4 builds on this by implementing the higher-level orchestration required to compute the hash for an entire message. Specifically, it covers the process of preparing the message, expanding it into a message schedule, initializing working variables, performing 64 rounds of compression, and combining the results into the evolving hash state, as defined in the Secure Hash Standard Section 6.

SHA-256 is a block-based hash function: it takes an input message of arbitrary length, divides it into fixed 512-bit blocks, and processes each block sequentially. To maintain cryptographic security, every bit of each message block must influence the final hash. Problem 4 ensures this through several carefully designed steps:

* Message schedule expansion: Each 512-bit block is split into 16 initial 32-bit words, which are then expanded to a 64-word message schedule. This ensures that information from the block affects all 64 rounds of compression
* Working Variables Initialisation: Eight 32-bit working variables are initialised from the current hash value. These variables evolve through 64 rounds of non-linear operations and bit mixing
* 64 Rounds of Compression: Using the message schedule, the round constants, and SHA-256 functions (Ch, Maj, Σ₀, Σ₁), the working variables are iteratively updated, propagating the influence of each bit through the hash state
* Hash State Update: After processing each block, the updated working variables are added to the current hash value to produce the intermediate hash. This chaining ensures that all blocks contribute to the final output

By combining these steps, Problem 4 produces a secure, deterministic, and irreversible hash value for any message. Five functions were implemented break down this process into manageable operations:

* expand_message_schedule(block) – Expands 16 message words into 64 words for compression
* initialise_working_variables(current_hash) – Sets up the working state from the current hash
* compress(working_vars, W, K) – Performs 64 rounds of SHA-256 compression using the message schedule and constants
* sha256_hash_block(current_hash, block) – Orchestrates the expansion, initialisation, and compression for a single block
* sha256(message) – Processes all blocks and returns the final 256-bit hash as a hexadecimal string

This structure mirrors the SHA-256 specification in FIPS 180-4, making it clear how each stage contributes to the security, diffusion, and one-way nature of the algorithm.

#### **Define Hash Values - H Constants**

Below are the eight 32-bit constants define the initial internal state of SHA-256. They are specified in NIST FIPS 180-4, Section 5.3.3. Each value is the first 32 bits of the fractional part of the square root of one of the first eight prime numbers. These numbers ensure transparency and prevent the introduction of hidden structures. All compliant SHA-256 implementations MUST use these exact values. The constants are stored as unsigned 32-bit integers to enforce arithmetic modulo 2³² throughout the algorithm.

In [210]:
H0 = np.array([
        0x6a09e667, 
        0xbb67ae85,  
        0x3c6ef372, 
        0xa54ff53a, 
        0x510e527f, 
        0x9b05688c, 
        0x1f83d9ab, 
        0x5be0cd19,  
    ], dtype=np.uint32)

#### **Define Hash Values - K Constants**

The array K contains the 64 fixed round constants used by the SHA-256 compression function and is defined by the Secure Hash Standard (FIPS 180-4, Section 4.2.2). One constant is injected into each of the 64 compression rounds to break symmetry between rounds and to ensure that identical message patterns do not evolve identically through the algorithm. Each constant K[t] is the first 32 bits of the fractional part of the cube root of the (t+1)th prime number, a construction that produces publicly verifiable numbers and prevents the introduction of hidden structures. The constants must be used in the exact order specified by the standard, stored as unsigned 32-bit integers, and treated as immutable, as any modification results in a non-compliant hash function.

In [211]:
import numpy as np

K = np.array([
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
], dtype=np.uint32)


### **expand_message_schedule(block) Function**

**Purpose** 

This function implements the message schedule expansion as defined in FIPS 180-4 Section 6.2.2. At this point in the SHA-256 process, the input message has been padded and divided into fixed-size 512-bit blocks by the functions defined in section 3 of this document. The compression function processes each block of the message and uses it to update the internal hash state by combining the current message block with the existing intermediate hash values. 

SHA-256 performs 64 rounds of compression, but each 512-bit block of the message only contains 16 words (32 bits each). This function expands those 16 words into 64 words, which ensures that information from the message block influences all 64 rounds of the compression function, rather than only the first 16 rounds. The first 16 words come directly from the message block, and the remaining 48 words are generated using the sigma0 and sigma1 functions to create thorough mixing of the input bits.

The expansion formula ensures that:

* Each input word influences multiple rounds
* Bits are mixed thoroughly via sigma0 and sigma1
* The avalanche effect propagates through all rounds
* Small message changes affect the entire schedule

**Endianness:** Endianness defines the order in which bytes are arranged within a multi-byte word. In big-endian, the most significant byte (the “big end”) comes first, while in little-endian, the least significant byte comes first. SHA-256 requires big-endian interpretation of message blocks, meaning the first byte of each 32-bit word is the most significant. This matters because the hash computation operates on 32-bit words, so interpreting the bytes in the wrong order (little-endian) would produce different word values, causing the resulting hash to be incorrect. In Python, this is ensured when reading the block with np.frombuffer(block, dtype='>u4'), where > specifies big-endian 32-bit unsigned integers.


**Implementation**

The expand_message_schedule() function ensures that every bit of the message eventually affects all rounds of the hash computation. To do this, it first interprets a message block as 16 'words' by dividing the 512-bit input block into 16 32-bit (4-byte) unsigned integers. This is done using np.frombuffer(block, dtype='>u4'), which reads the message block as consecutive 4-byte segments 'u4' interpreted as big-endian integers using '>'. These 16 'words' are stored in block_words and form the first part of the message schedule (W[0..15]). 

The next step in the message schedule expansion is to create an array 'W' to hold all 64 words of the schedule, which is done using NumPy's zeroes() function to create an array of 64 elements, all set to zero. Then the first 16 words in block_words are copied into the message schedule array 'W'.
The next step is to compute the remaining 48 words of the schedule, from W[16] to W[63], using the current 16 words stored in 'W' and the SHA-256 sigma functions. A loop iterates over indices 16 to 63, where for each index t, the value of W[t] is calculated by applying the sum of the following operations:

* sigma1(W[t-2]): the sigma1 function (defined in section 1)  is applied to the word two positions before t, which applies shift and rotations to mix the bits of W[t-2] in a complex way.
* W[t-7]: the word seven positions before t is added without modification. This step propagates information from earlier message words further into the schedule, ensuring that each original message word affects multiple rounds of the compression function.
* sigma0(W[t-15]): – the sigma0 function (defined in section 1) is applied to the word fifteen positions before t. Like sigma1, this function applies rotations and shifts to spread the influence of older words across the schedule which contributes to diffusion.
* W[t-16]: the word sixteen positions before t is added directly. This incorporates even older message content, making sure that all 16 original message words continue to influence the schedule.

The addition of these four components is implicitly modulo 2^32 because the W array has dtype np.uint32. This step ensures that any overflow wraps around, maintaining the fixed 32-bit word size. The results of all these operation are added together to produce a new 32-bit unsigned integer which is added to the array of word 'W'. 

Once the loop terminates, and 'W' has been populated with an additional 48 32-bit undigned integers, 'W' now constitutes the full message schedule array of 64 words. It is returned and will later be used in the compression function, with each word W[t] contributing to one round.

This expand_message_schedule() function implements these steps in the following way: 

* Step 1: Take the 512-bit block – Split it into 16 small pieces called words, each 32 bits (4 bytes) long
* Step 2: Create a schedule array – Make an empty array W with 64 spots to hold all the words for the hash computation.
* Step 3: Copy the first 16 words – Put the 16 original message words into the first 16 positions of W.
* Step 4: Compute the remaining 48 words – For each position from 16 to 63: Look at the words at positions t-2, t-7, t-15, and t-16 and apply sigma0 and sigma1 to t-2 and t-15. 
* Step 5: Add all four words together (wrapping around if the sum exceeds 32 bits) and store the result in W[t].
* Step 6: Return the full schedule (W) of 64 words

**Usage in SHA-256**

The expand_message_schedule(block) function is called once per 512-bit message block during the SHA-256 hash computation. Its output, the 64-word array W, is used directly in the compression function: each word W[t] is added during round t of the 64-round transformation.

**Relation to other functions**

* Used by: sha256_hash_block() – provides the 64-word message schedule required for the 64 compression rounds.
* Uses: sigma0() and sigma1() – non-linear functions that rotate and shift bits for diffusion.
* Feeds into: compress() – each word W[t] is combined with the working variables and round constants to update the hash state.
* Depends on: Proper parsing of the message block as 32-bit big-endian words (np.frombuffer(block, dtype='>u4')).

In [212]:
def expand_message_schedule(block):
    """
    Purpose: Expand a 512-bit message block into 64-word message schedule
    
    Parameters
    ----------
    block : bytes
        512-bit (64-byte) message block
        
    Returns
    -------
    numpy.ndarray of np.uint32
        Array of 64 words (W₀ through W₆₃), shape (64,)

    """
    # Parse 512-bit block as array of 16 big-endian 32-bit unsigned integers
    block_words = np.frombuffer(block, dtype='>u4')
    
    # Create array to hold 64-word message schedule
    W = np.zeros(64, dtype=np.uint32)
    
    # Get first 16 words directly from the message block & store them in W
    for t in range(16):
        W[t] = block_words[t]
    
    # Generate remaining 48 words using expansion formula 
    for t in range(16, 64):
        W[t] = sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16]
    
    # return message schedule
    return W

### **initialise_working_variables(current_hash) Function**

**Purpose**

The initialise_working_variables(current_hash) function sets up the eight 32-bit working variables used in the SHA-256 compression function. These eight working variables (a, b, c, d, e, f, g, h) form the internal state for the 64-round compression process. By initializing them with the current hash, the function ensures that each block’s processing builds upon all previously processed blocks, creating a chain of dependencies known as chaining values. This makes it impossible to alter earlier blocks without affecting the final hash and preserves the cryptographic property of diffusion, where a change in any input bit eventually affects all output bits.

**Implementation**

This function implements Step 2 of the SHA-256 algorithm as defined in FIPS 180-4 Section 6.2.2: it extracts each 32-bit word from the current_hash array and assigns it to a distinct working variable. These working variables will then be iteratively updated during the 64-round compression process.

* Step 1: Extract H0 through H7 by reading each element of the current_hash array, which contains 8 32-bit unsigned integers representing the current intermediate hash value. Each extracted hash value is assigned to one of the working variables (a, b, c, d, e, f, g, h) in the same order as specified in the standard
* Step 2: The function returns the tuple (a, b, c, d, e, f, g, h) ready for use in the 64-round compression function.

**Usage in SHA-256**

initialise_working_variables() is called once per 512-bit message block, immediately before the 64 compression rounds begin. The returned variables (a, b, c, d, e, f, g, h) are updated iteratively in the compress() function.

**Relation to other functions**

* Used by: sha256_hash_block() to provide the starting state for compression rounds
* Feeds into: compress() where the working variables are iteratively updated during all 64 rounds

In [213]:
def initialise_working_variables(current_hash):
    """
    Purpose: Initialize eight working variables from current hash value.
    
    Parameters
    ----------
    current_hash : numpy.ndarray of np.uint32
        Current hash values H₀ through H₇
        
    Returns
    -------
    tuple of np.uint32
        Eight working variables (a, b, c, d, e, f, g, h)
        
    """
    # Extract hash values into eight separate variables
    a = current_hash[0]  # H0
    b = current_hash[1]  # H1
    c = current_hash[2]  # H2
    d = current_hash[3]  # H3
    e = current_hash[4]  # H4
    f = current_hash[5]  # H5
    g = current_hash[6]  # H6
    h = current_hash[7]  # H7
    
    # return the variables 
    return a, b, c, d, e, f, g, h

### **compress(working_vars, W, K) Function**

**Purpose**

The compress() function performs the 64-round SHA-256 compression operation, as defined in FIPS 180-4 Section 6.2.2, Step 3. At this stage, the padded message has been split into 512-bit blocks, and a message schedule W has been prepared. The compression function takes the current working variables (a, b, c, d, e, f, g, h) and iteratively mixes them with the message schedule W and round constants K over 64 rounds. Each round applies non-linear functions, bitwise rotations, and additions to thoroughly diffuse the message. This process ensures that small changes in the message or initial state propagate to all bits of the output hash, creating the avalanche effect. The output of this function is a new set of working variables that will later be combined with the current hash to produce the next intermediate hash.

**Implementation**

This compress() function uses the 64-round iterative process to update the variables through a combination of linear additions and non-linear transformations. To do this, the function first takes in and unpacks working variables by extracting the initial values of (a, b, c, d, e, f, g, h) from the input tuple. Each variable is a 32-bit unsigned integer that represents a portion of the intermediate hash state. Next, a loop iterates through 64 rounds where all working variables are updated using both the message schedule W and the round constants K. These updates are done using functions from section 1: Sigma0(), Sigma1(), Maj(), and Ch(). Each round is performed as follows:

First, T1 is calculated: T1 = h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]

* h carries the previous round’s state, propagating the cascade of transformations
* Sigma1(e) rotates and shifts the bits of e in a non-linear way to mix its contribution across all bits
* Ch(e, f, g) selects bits from f or g based on the value of e. This is a conditional, non-linear operation that ensures the transformation is not reversible
* K[t] introduces a unique constant for each round, breaking symmetry and further increasing diffusion
* W[t] incorporates the message schedule word for this round, ensuring that every word of the input message influences the output hash
* All additions are performed modulo 2³², ensuring that each T1 remains a 32-bit unsigned integer

Next, T2 is calculated: T2 = Sigma0(a) + Maj(a, b, c)

* Sigma0(a) performs right rotations and shifts on a, spreading its influence across all bits
* Maj(a, b, c) computes the majority function of a, b, c, effectively selecting the value that occurs in at least two of the three variables. This adds a non-linear component that mixes the state

Finally, all working variables are updated using T1 and T2 as follows:

h ← g

g ← f

f ← e

e ← d + T1

d ← c

c ← b

b ← a

a ← T1 + T2

This cascading effect ensures that each variable in the next round depends on all the variables from the previous round, which propagates every bit’s influence throughout all 64 rounds

Once all 64 rounds are complete and the loop ends, the working variables have been thoroughly transformed in a way that combines message-dependent and state-dependent information. The updated working variables are returned as a tuple, which will later be added to the current intermediate hash value to create the updated hash for this message block.

**Usage in SHA-256**

The compress() function is called once per 512-bit block of the message and produces a set of transformed working variables that are added back to the intermediate hash to update the hash state

**Relation to other functions**

* Uses: Ch(), Maj(), Sigma0(), Sigma1(), message schedule W, and round constants K.
* Called by: sha256_hash_block().
* Provides: Updated working variables that contribute to the new hash state

In [214]:
def compress(working_vars, W, K):
    """
    Purpose: Implements the core of SHA-256 by performing 64 rounds of compression
    
    Parameters
    ----------
    working_vars : tuple of np.uint32
        Initial working variables (a, b, c, d, e, f, g, h)
    W : numpy.ndarray of np.uint32
        64-word message schedule
    K : numpy.ndarray of np.uint32
        64 round constants
        
    Returns
    -------
    tuple of np.uint32
        Updated working variables after 64 rounds (a, b, c, d, e, f, g, h)

    """
    # Unpack working variables
    a, b, c, d, e, f, g, h = working_vars
    
    # Perform 64 rounds of compression
    for t in range(64):
        # T1 combines h with transformations on e, f, g
        T1 = h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]
        
        # T2 combines transformations on a, b, c
        T2 = Sigma0(a) + Maj(a, b, c)
        
        # Update working variables (cascade effect)
        # Each variable receives value from "upstream" variable
        # This creates a flow of information through all variables
        h = g              
        g = f              
        f = e              
        e = d + T1         
        d = c              
        c = b              
        b = a              
        a = T1 + T2        
    
    # Return updated working variables after all 64 rounds
    return a, b, c, d, e, f, g, h

### **sha256_hash_block(current_hash, block) Function**

**Purpose**

The sha256_hash_block() function implements the complete SHA-256 computation for a single 512-bit message block, as defined in FIPS 180-4 Section 6.2.2. It takes the current intermediate hash value and a message block, processes the block using the expand_message_schedule() and compress() functions, and produces an updated hash state. This updated state incorporates the new message block while preserving all the effects of previous blocks, ensuring that each bit of the message influences the final hash in a cryptographically secure way. By sequentially processing all blocks with this function, the SHA-256 algorithm builds a final 256-bit hash that is deterministic, irreversible, and collision-resistant.

This function guarantees:

* Every block influences the hash through a complex, non-linear transformation
* Earlier blocks propagate their effect to later blocks via the working variables and chaining mechanism
* The avalanche effect ensures that a single-bit change in the message results in a completely different hash
* The output depends on every bit of the input message and is suitable for cryptographic purposes

**Implementation**

The sha256_hash_block() function computes the new intermediate hash by first taking in the current hash calculated by the compress() function, as well as the next block of the message. Next, the function computes the message schedule 'W' by calling the expand_message_schedule() function defined previously in this section. The working variables are then initialised using the initialise_working_variables() function (defined previously), and these variables are input into the compress() (defined previously) to obtain the next set of working variables after 64 rounds of compression. Next, this function computes the new intermediate hash value by adding the final working variables (a, b, c, d, e, f, g, h) to the corresponding words of current_hash, and storing these resulting values in a uint32 array which is then returned. The sha256_hash_block() function implements these steps in the following manner:

* Step 1: Expand the message schedule
* Step 2: Initialise the working variables
* Step 3: Perform 64 rounds of compression
* Step 4: Compute the new intermediate hash value 
* Step 5: Return the new intermediate hash value

**Usage in SHA-256**

This function is the heart of SHA-256. It takes the current hash state and a message block, and produces an updated hash state that incorporates the information from that block. By processing all blocks sequentially, it builds up the final hash value that uniquely represents the message.

**Relation to other function**

* Uses: expand_message_schedule() to create the 64-word message schedule, initialise_working_variables() to set up the working variables, and compress() to perform the 64-round transformation
* Called by: The main sha256() function
* Provides: Updated intermediate hash value for the next block or final output

In [215]:
def sha256_hash_block(current_hash, block):
    """
    Compute next hash value from current hash and message block by 
    orchestrating all previous steps of the hash computation
    
    Parameters
    ----------
    current_hash : numpy.ndarray of np.uint32
        Current hash value H, contains eight 32-bit words H0 -> H7
    block : bytes
        512-bit (64-byte) message block M
        
    Returns
    -------
    numpy.ndarray of np.uint32
        Updated hash value H
  
    """
    # Prepare the message schedule (expand 16 words to 64)
    W = expand_message_schedule(block)
    
    # Initialise the eight working variables with current hash
    a, b, c, d, e, f, g, h = initialise_working_variables(current_hash)
    
    # Perform 64 rounds of compression
    a, b, c, d, e, f, g, h = compress((a, b, c, d, e, f, g, h), W, K)
    
    # Compute the intermediate hash value by adding the working variables to the current hash (modulo 2³²)
    new_hash = np.array([
        a + current_hash[0],  
        b + current_hash[1],  
        c + current_hash[2],  
        d + current_hash[3],  
        e + current_hash[4],  
        f + current_hash[5],  
        g + current_hash[6],  
        h + current_hash[7],  
    ], dtype=np.uint32)
    
    # return the new intermediate hash value array
    return new_hash

### **sha256(message) Function**

**Purpose**

This function implements the full SHA-256 algorithm for messages of any length. It orchestrates the preprocessing, block parsing, compression, and final formatting steps to produce a cryptographically secure 256-bit hash value. By processing each 512-bit block sequentially and updating the intermediate hash state, it ensures that every bit of the input message influences the final output. The returned hash is represented as a 64-character hexadecimal string.

**Implementation**

This function implements the full SHA-256 algorithm and produces a 256-bit hash as 64-character hexadecimal string

* Step 1: Initialize the hash state by copying the initial constants H0 (H0 through H7), which are defined by the SHA-256 standard
* Step 2: Parse the message into 512-bit blocks using the block_parse(message) function defined in section 3
* Step 3: Get the final hash values by processing each block using the sha256_hash_block(block) function, which coordinates all the functions that perform: message schedule expansion, working variable calculation, and compression, and updates the current hash value by adding the resulting working variables modulo 2³².
* Step 4: Format the final hash by converting the 8-word final hash value (H₀ through H₇) into a 64-character hexadecimal string. The concatenation of all 8 words produces the final 256-bit hash.

In [216]:
def sha256(message):
    """
    Complete SHA-256 hash computation.
    
    Parameters
    ----------
    message : bytes
        Message to hash (any length)
        
    Returns
    -------
    str
        256-bit hash as 64-character hexadecimal string
    """

    # copy the first 8 H constant
    H = H0.copy()
    
    # Process each block
    for block in block_parse(message):
        H = sha256_hash_block(H, block)
    
     # Format as hex string
    hex_hash = ''.join(f'{x:08x}' for x in H)

    # return hash as a hex string
    return hex_hash

### **Problem 4 Testing**

#### **Testing expand_message_schedule()**

In [217]:
def test_expand_message_schedule():
    print("\n" + "=" * 72)
    print("TEST 1: expand_message_schedule()")
    print("=" * 72)

    block = b"A" * 64
    print(f"Input block (first 16 bytes): {block[:16]}")
    print(f"Block hex (first 32 chars):  {block.hex()[:32]}")

    W = expand_message_schedule(block)

    print("\nMessage schedule created")
    print(f"Type: {type(W)}, dtype: {W.dtype}, length: {len(W)}")

    print("\nFirst 16 words (direct from block):")
    for i in range(16):
        print(f"W[{i:2d}] = 0x{W[i]:08x}")

    print("\nExpanded words:")
    for i in (16, 17, 18, 32, 48, 63):
        print(f"W[{i:2d}] = 0x{W[i]:08x}")

    assert isinstance(W, np.ndarray)
    assert W.dtype == np.uint32
    assert len(W) == 64

    print("\nAvalanche effect check:")
    block2 = bytearray(block)
    block2[0] ^= 0x01

    W2 = expand_message_schedule(bytes(block2))
    differences = np.sum(W != W2)

    print(f"Words changed after 1-bit flip: {differences}/64")
    assert differences > 0

    print("expand_message_schedule PASSED")


test_expand_message_schedule()


TEST 1: expand_message_schedule()
Input block (first 16 bytes): b'AAAAAAAAAAAAAAAA'
Block hex (first 32 chars):  41414141414141414141414141414141

Message schedule created
Type: <class 'numpy.ndarray'>, dtype: uint32, length: 64

First 16 words (direct from block):
W[ 0] = 0x41414141
W[ 1] = 0x41414141
W[ 2] = 0x41414141
W[ 3] = 0x41414141
W[ 4] = 0x41414141
W[ 5] = 0x41414141
W[ 6] = 0x41414141
W[ 7] = 0x41414141
W[ 8] = 0x41414141
W[ 9] = 0x41414141
W[10] = 0x41414141
W[11] = 0x41414141
W[12] = 0x41414141
W[13] = 0x41414141
W[14] = 0x41414141
W[15] = 0x41414141

Expanded words:
W[16] = 0xe6165654
W[17] = 0xe6165654
W[18] = 0x3f56e7d8
W[32] = 0x83f29ac2
W[48] = 0x226ad9e3
W[63] = 0x3bf5d4c7

Avalanche effect check:
Words changed after 1-bit flip: 46/64
expand_message_schedule PASSED


#### **Testing initialise_working_variables()**

In [218]:
def test_initialise_working_variables():
    print("\n" + "=" * 72)
    print("TEST 2: initialise_working_variables()")
    print("=" * 72)

    H = H0.copy()
    print("\nInitial hash values:")
    for i, v in enumerate(H):
        print(f"H[{i}] = 0x{v:08x}")

    a, b, c, d, e, f, g, h = initialise_working_variables(H)

    print("\nWorking variables:")
    print(f"a={a:08x}  b={b:08x}  c={c:08x}  d={d:08x}")
    print(f"e={e:08x}  f={f:08x}  g={g:08x}  h={h:08x}")

    assert a == H[0]
    assert b == H[1]
    assert c == H[2]
    assert d == H[3]
    assert e == H[4]
    assert f == H[5]
    assert g == H[6]
    assert h == H[7]

    print("initialise_working_variables PASSED")

test_initialise_working_variables()


TEST 2: initialise_working_variables()

Initial hash values:
H[0] = 0x6a09e667
H[1] = 0xbb67ae85
H[2] = 0x3c6ef372
H[3] = 0xa54ff53a
H[4] = 0x510e527f
H[5] = 0x9b05688c
H[6] = 0x1f83d9ab
H[7] = 0x5be0cd19

Working variables:
a=6a09e667  b=bb67ae85  c=3c6ef372  d=a54ff53a
e=510e527f  f=9b05688c  g=1f83d9ab  h=5be0cd19
initialise_working_variables PASSED


#### **Testing compress()**

In [219]:
def test_compress():
    print("\n" + "=" * 72)
    print("TEST 3: compress()")
    print("=" * 72)

    block = b"A" * 64
    W = expand_message_schedule(block)
    working_vars = initialise_working_variables(H0.copy())

    print("\nInitial working variables:")
    labels = "abcdefgh"
    for lbl, val in zip(labels, working_vars):
        print(f"{lbl} = 0x{val:08x}")

    print("\n--- Tracing round 0 ---")
    a, b, c, d, e, f, g, h = working_vars

    T1 = h + Sigma1(e) + Ch(e, f, g) + K[0] + W[0]
    T2 = Sigma0(a) + Maj(a, b, c)

    print(f"T1 = 0x{T1:08x}")
    print(f"T2 = 0x{T2:08x}")
    print(f"New a = T1 + T2 = 0x{(T1 + T2):08x}")
    print(f"New e = d + T1 = 0x{(d + T1):08x}")

    result = compress(working_vars, W, K)

    print("\nFinal working variables after 64 rounds:")
    for lbl, val in zip(labels, result):
        print(f"{lbl} = 0x{val:08x}")

    assert isinstance(result, tuple)
    assert len(result) == 8

    print("\nSensitivity test (flip 1 bit in a):")
    modified = list(working_vars)
    modified[0] ^= 1
    modified = tuple(modified)

    result2 = compress(modified, W, K)

    diff = sum(x != y for x, y in zip(result, result2))
    print(f"Different outputs: {diff}/8")

    assert diff > 0

    print("compress PASSED")

test_compress()


TEST 3: compress()

Initial working variables:
a = 0x6a09e667
b = 0xbb67ae85
c = 0x3c6ef372
d = 0xa54ff53a
e = 0x510e527f
f = 0x9b05688c
g = 0x1f83d9ab
h = 0x5be0cd19

--- Tracing round 0 ---
T1 = 0x34b92ea9
T2 = 0x08909ae5
New a = T1 + T2 = 0x3d49c98e
New e = d + T1 = 0xda0923e3

Final working variables after 64 rounds:
a = 0x02ad3dda
b = 0x478ea90b
c = 0xd7e24e6f
d = 0x6082ce99
e = 0x3693869f
f = 0x01e838ae
g = 0x2ab94c32
h = 0x2bde1cec

Sensitivity test (flip 1 bit in a):
Different outputs: 8/8
compress PASSED


#### **Testing sha256_hash_block()**

In [220]:
def test_sha256_hash_block():
    print("\n" + "=" * 72)
    print("TEST 4: sha256_hash_block()")
    print("=" * 72)

    H = H0.copy()
    block = b"A" * 64

    print("\nInitial hash:")
    for i, v in enumerate(H):
        print(f"H[{i}] = 0x{v:08x}")

    H_new = sha256_hash_block(H, block)

    print("\nUpdated hash:")
    for i, v in enumerate(H_new):
        print(f"H[{i}] = 0x{v:08x}")

    assert not np.array_equal(H, H_new)

    print("\nChaining second block...")
    H2 = sha256_hash_block(H_new, block)

    print(f"H[0] after second block = 0x{H2[0]:08x}")
    assert not np.array_equal(H_new, H2)

    print("sha256_hash_block PASSED")

test_sha256_hash_block()


TEST 4: sha256_hash_block()

Initial hash:
H[0] = 0x6a09e667
H[1] = 0xbb67ae85
H[2] = 0x3c6ef372
H[3] = 0xa54ff53a
H[4] = 0x510e527f
H[5] = 0x9b05688c
H[6] = 0x1f83d9ab
H[7] = 0x5be0cd19

Updated hash:
H[0] = 0x6cb72441
H[1] = 0x02f65790
H[2] = 0x145141e1
H[3] = 0x05d2c3d3
H[4] = 0x87a1d91e
H[5] = 0x9ceda13a
H[6] = 0x4a3d25dd
H[7] = 0x87beea05

Chaining second block...
H[0] after second block = 0xdfdaa928
sha256_hash_block PASSED


#### **Testing sha256()**

In [221]:
def test_sha256_complete():
    print("\n" + "=" * 72)
    print("TEST 5: sha256()")
    print("=" * 72)

    vectors = {
        b"abc":
            "ba7816bf8f01cfea414140de5dae2223"
            "b00361a396177a9cb410ff61f20015ad",
        b"":
            "e3b0c44298fc1c149afbf4c8996fb924"
            "27ae41e4649b934ca495991b7852b855",
    }

    for msg, expected in vectors.items():
        print(f"\nMessage: {msg}")
        result = sha256(msg)
        print(f"Result:   {result}")
        print(f"Expected: {expected}")
        assert result == expected

    print("\nAvalanche effect:")
    h1 = sha256(b"The quick brown fox jumps over the lazy dog")
    h2 = sha256(b"The quick brown fox jumps over the lazy cog")

    diff = sum(a != b for a, b in zip(h1, h2))
    print(f"Differing hex characters: {diff}/64")

    assert diff > 30

    print("sha256 PASSED")

test_sha256_complete()


TEST 5: sha256()

Message: b'abc'
Result:   ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Expected: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad

Message: b''
Result:   e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Expected: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855

Avalanche effect:
Differing hex characters: 59/64
sha256 PASSED


## Problem 5: Passwords

**Brief:** The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. As strings, they were encoded using UTF-8. Determine the passwords and explain how you found them. Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

1. 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
2. 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
3. b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

### **Problem 5 Introduction**

This problem examines a fundamental and widely exploited security weakness in password encryption: the use of unsalted cryptographic hash functions. Although SHA-256 is a secure and widely trusted cryptographic hash algorithm, if not used correctly for password storage, SHA-256 can leave systems vulnerable to offline attacks.
In this scenario, three passwords have been hashed using a single application of the SHA-256 algorithm and stored without any additional protective mechanisms such as salting, key stretching, or multiple hashing rounds. The absence of these defenses significantly reduces the effort required to recover the original passwords, especially when users select weak or commonly used credentials. Three passwords have been hashed using a single pass of SHA-256 and stored without any additional security measures. This section of the notebook addresses the three following topics:

1. **Cracking the passwords** - Determining the original passwords from their hashes
2. **Explaining the method** - Documenting how the attack was performed
3. **Suggesting improvements** - Recommended security measures to prevent such attacks

The solution will be implemented based on the information provided:

* Three hashes of common passwords
* The passwords were encoded using UTF-8 before hashing 
* Only one pass of SHA-256 was used
* No salt or additional security measures were applied

Something important to note is that unsalted passwords are deterministic, meaning that the same passwords always produce the same hashes.
Given this information, the most logical solution to decrypting the three provided hashes is to apply a dictionary attack using the implementation of the SHA-256 algorithm shown in this notebook.

**Dictionary attack** 

This is an attack which systematically tries common passwords from a predefined list (dictionary/wordlist) until finding a match. This exploits human tendency to choose:
- Common words ("password", "welcome")
- Simple patterns ("123456", "qwerty")
- Personal information (names, dates)
- Slight variations ("password1", "P@ssw0rd")

In this section, a dictionary attack will systematically test each entry from a predefined list of common passwords in order to decrypt the provided hashes. To do this, the following steps will be implemented:

* Encoding each candidate password using UTF-8.
* Hashing the encoded password using the same SHA-256 implementation (the one implemented in this notebook)
* Comparing the resulting hash to the target hash
* Identifying a match when the computed hash equals the stored hash

These steps are implemented using the functions in this section:

* crack_password_hash(target_hash, passwordList): 
* crack_all_passwords():

#### Top 100 most common passwords - from NORDVPN https://nordpass.com/most-common-passwords-list/

In [222]:
# list of with very common passwords
passwordList = [
    "password", "123456", "12345678", "qwerty", "abc123",
    "monkey", "1234567", "letmein", "trustno1", "dragon",
    "baseball", "iloveyou", "master", "sunshine", "ashley",
    "bailey", "P@ssw0rd", "cheese", "123123", "654321",
    "superman", "qazwsx", "michael", "football", "password1"
]

#### The three hashes from the problem

In [223]:
hashes = {
    "Hash 1": "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "Hash 2": "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "Hash 3": "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
}

### **crack_password_hash(target_hash, passwordList) Function**

**Purpose**

The purpose of the crack_password_hash() function is to recover a plaintext password from a given SHA-256 hash by performing a dictionary attack. It systematically hashes a list of commonly used passwords using the sha-256 algorithm implemented in previous sections, and compares each computed hash to the target hash. If a match is found, the corresponding plaintext password is returned.
This function demonstrates how weak password choices and unsalted hashing make password hashes vulnerable to offline attacks, even when a cryptographically strong hash function such as SHA-256 is used.

**Implementation**

The function operates as follows:

* Step 1: A counter 'attempts' is initialised to track how many password guesses have been tested. This provides insight into how quickly a password can be cracked using a small passwordList.
* Step 2: The function loops through each password in the provided passwordList, treating each entry as a potential plaintext password.
* Step 3: Each password string is encoded into UTF-8 bytes. This step is required because the SHA-256 hashing algorithm operates on byte data rather than Python strings
* Step 4: The encoded password is hashed using a custom SHA-256 implementation developed earlier in the project. This ensures the hashing process exactly matches the one used to generate the target hashes.
* Step 5: The computed hash is compared directly to the target hash. Because SHA-256 is deterministic and no salt was used, identical passwords will always produce identical hashes.
* Step 6: If a match is found the function prints the number of attempts required and the plaintext password is returned immediately. If all candidate passwords are tested without finding a match the function reports the total number of attempts and returns 'None', indicating that the password was not found in the passwordList.

In [224]:
def crack_password_hash(target_hash, passwordList):
    """
    Attempt to crack a SHA-256 password hash using a list of the top common passwords
    
    Parameters
    ----------
    target_hash : str
        The SHA-256 hash to crack (64 hex characters)
    passwordList : list of str
        List of potential passwords to try
        
    Returns
    -------
    str or None

        The cracked password if found, None otherwise
    """
    attempts = 0
    
    # Iterate over each passwords stored in the passwordList
    for password in passwordList:
        
        # Encode password as UTF-8 bytes
        password_bytes = password.encode('utf-8')
        
        # Hash the password using the SHA-256 implementation from section 4
        computed_hash = sha256(password_bytes)
        
        attempts += 1
        
        # Check if it matches
        if computed_hash == target_hash:
            print(f"Password found after {attempts} attempts!")
            print(f"Target hash  : {target_hash}")
            print(f"Computed hash: {computed_hash}")
            return password
    
    print(f"Password not found after {attempts} attempts")
    
    return None

### **crack_all_passwords() Function**

**Purpose**

The crack_all_passwords function coordinates the cracking of multiple password hashes by repeatedly invoking crack_password_hash. It serves as a driver function that automates the attack against all provided hashes and aggregates the results.
This function demonstrates how dictionary attacks can be scaled efficiently to compromise multiple accounts when unsalted hashes are used.

**Implementation**

The function performs the following steps:
* Step 1: A dictionary named results is created to store the cracked passwords, mapping each hash identifier (e.g., "Hash 1") to its recovered plaintext password or 'None'
* Step 2: The function iterates over a dictionary of named hashes. For each hash the hash name and value are printed and the crack_password_hash function is called to check each hash against the hashed passwords from the passwordList
* Step 3: If a password is successfully cracked, it is printed. If no password is found, the function reports the failure.
* Step 4: After all hashes have been processed, the function ends

In [225]:
def crack_all_passwords():
    """
    Crack all three password hashes from the problem.
    """
    print("=" * 70)
    print("SHA-256 PASSWORD CRACKING")
    print("=" * 70)
    print()
    
    # iterate through the hashes dictionary to find a matching password for each hash
    for name, target_hash in hashes.items():
        print(f"Cracking {name}:")
        
        # Check if each hash matches a passowrd in the passwordList
        password = crack_password_hash(target_hash, passwordList)
        
        # If a passowrd is found, print it; otherwise indicate not found
        if password:
            print(f"Password: '{password}'")
        else:
            print(f"Password: NOT FOUND in wordlist")
        print()

In [226]:
print("\n")
print("╔" + "=" * 68 + "╗")
print("║" + " " * 20 + "PROBLEM 5: PASSWORD CRACKING" + " " * 20 + "║")
print("╚" + "=" * 68 + "╝")
print("\n")
    
# Crack the passwords
crack_all_passwords()




╔====================================================================╗
║                    PROBLEM 5: PASSWORD CRACKING                    ║
╚====================================================================╝


SHA-256 PASSWORD CRACKING

Cracking Hash 1:
Password found after 1 attempts!
Target hash  : 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Computed hash: 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Password: 'password'

Cracking Hash 2:
Password found after 18 attempts!
Target hash  : 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Computed hash: 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Password: 'cheese'

Cracking Hash 3:
Password found after 17 attempts!
Target hash  : b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
Computed hash: b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342
Password: 'P@ssw0rd'



#### **Problem 5 conclusion**

The successful recovery of all three passwords demonstrates a fundamental weakness in using a single-pass, unsalted cryptographic hash for password storage. Although SHA-256 is cryptographically secure for integrity verification, it is not designed to resist high-speed brute-force or dictionary attacks.

To prevent this type of attack, several improvements to password hashing should be implemented: 

* First, a unique cryptographic salt should be added to each password before hashing, ensuring that identical passwords produce different hashes and preventing the use of precomputed hash tables. 
* Second, key-stretching techniques such as multiple hashing iterations should be applied to significantly increase the computational cost of each password guess. 
* Finally, modern password-hashing algorithms specifically designed for this purpose—such as bcrypt, scrypt, or Argon2—should be used, as they are intentionally slow and resistant to parallel attacks.

## **Conclusion**

Summary of Achievement
This project successfully implemented the complete SHA-256 cryptographic hash function by following the FIPS PUB 180-4 specification. Through five progressive problems, we constructed a working implementation that:

* Passes all official test vectors (verified against NIST standards)
* Implements every specification detail (bitwise operations, constants, padding, compression)
* Demonstrates cryptographic properties (avalanche effect, determinism, irreversibility)
* Analyzes security implications (proper vs improper usage)
* Provides educational value (comprehensive documentation and testing)

**1. Problem 1: Cryptographic Primitives**
We began with the smallest building blocks — individual bitwise operations that seem simple in isolation but combine to create complex, non-linear transformations. The rotation, choice, and majority functions taught us that security emerges from layering simple operations in specific ways.

* Key insight: Non-linearity is essential. Linear operations, no matter how many, can be reduced to a single linear operation. The Ch and Maj functions introduce irreducible complexity.

**2. Problem 2: Cryptographic Constants**
By deriving 64 round constants from cube roots of prime numbers, we saw how transparency builds trust. These numbers can be independently verified by anyone, ensuring no backdoors exist.

* Key insight: In cryptography, trust comes from openness. Secret algorithms are vulnerable to hidden flaws; public algorithms withstand global scrutiny.

**3. Problem 3: Message Preprocessing**
Padding and parsing demonstrated that even data preparation involves security considerations. The specific padding scheme prevents extension attacks and ensures every message has a unique representation.

* Key insight: Security requires attention to every detail. The padding's length encoding isn't arbitrary—it's essential for preventing attacks.

**4. Problem 4: Hash Computation**
The compression function revealed the heart of SHA-256: 64 rounds of carefully orchestrated operations that transform a 512-bit block into a 256-bit hash. Each round depends on previous rounds, creating a cascade of dependencies impossible to reverse.

* Key insight: Cryptographic strength comes from iteration. One round might be analyzable; 64 rounds create impenetrable complexity.

**5. Problem 5: Security Analysis**
Password cracking exposed a crucial lesson: cryptographically secure does not equal secure for all purposes. SHA-256 is perfect for data integrity but dangerously inadequate for password storage.

* Key insight: Context matters. Using the right algorithm for the wrong purpose creates vulnerability, not security.